In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.cm as cm
import networkx as nx
import plotly as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc
import kaleido
import pingouin as pg
from scipy.stats import rayleigh, skew, kurtosis, pearsonr, genhyperbolic, laplace, spearmanr, ttest_ind
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy.spatial.distance import cdist
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter, welch, butter, filtfilt, freqz
from scipy.spatial.transform import Rotation as R
from scipy import stats
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import itertools
import os
import fnmatch
import ast

In [ ]:
def butter_lowpass_filter(data):
    cutoff = 6
    fs = 30
    order = 5
    nyq = 0.5 * fs
    nml_cutoff = cutoff / nyq
    b, a = butter(order, nml_cutoff, btype='low', analog=False)
    data_filtered = filtfilt(b, a, data)
    return data_filtered

def filterPositionData(df, center, right, subject):
    #print(np.min(df['Wrist_X']))
    #print(np.min(df['Wrist_Y']))
    #print(np.max(df['Wrist_Z']))
    joint_columns = [col for col in df.columns]
    for col in joint_columns:
        df[col] = butter_lowpass_filter(df[col].values)
    
    activityWrist = df[['Wrist_X', 'Wrist_Y', 'Wrist_Z']].values  # Just use wrist data to eliminate outliers - most noisy probably

    distances = np.linalg.norm(activityWrist, axis=1)  # Create vector of distances to center
    filtered_distances = distances[~np.isnan(distances)] # get rid of nan distances


    #get height from clinical files spreadsheet
    if subject[0] == 's':
        heightWeight_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase0_Heightweight_DATA_LABELS_2024_06_11_1650.csv')
    else:
        heightWeight_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase1-Heightweight_DATA_LABELS_2024_06_11_1654.csv')
    
    subject_Height = heightWeight_df[heightWeight_df['Record ID'].str.lower() == subject.lower()]['Height (inches)'] * 0.0254
    subject_Height = subject_Height.values[0]
    max_arm_length = subject_Height * 0.5 #based on DA winter's arm segment lengths (0.186, 10.146, 0.108 = 0.44 + wiggle room)
    #print('Max Arm Length', max_arm_length)

    #build a new array with only the values where distance is less than max arm length
    filtered_distances = np.array(filtered_distances)
    print("before distance filter", df.shape)
    df_filtered = df[filtered_distances < max_arm_length]
    print("after distance filter", df_filtered.shape)
    
    return df_filtered

# Function to check if a point is inside a pyramid
def is_inside_pyramid(point, pyramid_vertices, pyramid_number, pyramid_forceList, point_number, forces, IdxByPyramid, radius):
    # Check if the point is inside the pyramid using the cross and dot product product 
    v0, v1, v2, v3 = pyramid_vertices #v0 is the apex, v1,v2,v3 are base vertices
    side_vectors = [v1 - v0, v2 - v0, v3 - v0]
    point_vector = point - v0

    cross1 = np.cross(side_vectors[0], side_vectors[1])
    cross2 = np.cross(side_vectors[1], side_vectors[2])
    cross3 = np.cross(side_vectors[2], side_vectors[0])
    
    #check if these normal vectors point inwards, change if they do
    if np.dot(cross1, v3 - v0) < 0: cross1 = -cross1
    if np.dot(cross2, v1 - v0) < 0: cross2 = -cross2
    if np.dot(cross3, v2 - v0) < 0: cross3 = -cross3
        
    #compute dot products for each: if direction is same between point_vector and cross vector pointing in, point is in the zone
    dot1 = np.dot(point_vector, cross1)
    dot2 = np.dot(point_vector, cross2)
    dot3 = np.dot(point_vector, cross3)
    

    if dot1 < 0 or dot2 < 0 or dot3 < 0:
        return False
    else:
        if point_number < (len(forces) - 1):
            distance_fromShoulder = np.linalg.norm(point_vector)
            if distance_fromShoulder < (radius/2):
                pyramid_forceList[pyramid_number].append(forces[point_number-1])
                IdxByPyramid[pyramid_number].append(point_number)
            else:
                pyramid_forceList[pyramid_number+20].append(forces[point_number-1])
                IdxByPyramid[pyramid_number+20].append(point_number)
        return True
    
    
def plot_icosahedron(unique_pyramids, points_in_pyramid, wristMotionOnly, subject):
    fig = plt.figure(figsize=(10, 10), dpi=600)
    colors = generate_colors(points_in_pyramid)
    # Plot for the anterior view
    r=1
    transparency = 0.7
    ax1 = fig.add_subplot(121, projection='3d')
    plot_view(ax1, unique_pyramids, colors, r, transparency, elev=88, azim=270)

    #posterior is positiveZ
    #anterior is negativeZ
    negative_z = wristMotionOnly[wristMotionOnly[:,2] < 0]
    print(negative_z.shape)
    non_negative_z = wristMotionOnly[wristMotionOnly[:,2] >= 0]
    print(non_negative_z.shape)

    # Add scatter plot for the anterior view
    ax1.scatter(non_negative_z[:, 0], non_negative_z[:, 1], non_negative_z[:, 2], color='black', s=0.1)  # Adjust color and size as needed

    # Plot for the posterior view
    r += 1
    ax2 = fig.add_subplot(122, projection='3d')
    plot_view(ax2, unique_pyramids, colors, r, transparency, elev=-88, azim=90)
    #add scatter plot for posterior view
    ax2.scatter(negative_z[:, 0], negative_z[:, 1], negative_z[:, 2], color='black', s=0.1)  # Adjust color and size as needed


    #Save figure in desired path
    filepath
    directoryFigs = '/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/FiguresCollection/'
    Figs_filename = f'IcosahedronPlot_{subject}'
    filepathFigs = os.path.join(directoryFigs, Figs_filename)
    plt.savefig(filepathFigs, dpi=1600, bbox_inches='tight')
    plt.show()

def plot_view(ax, unique_pyramids, colors, r, transparency, elev, azim):
    ax.view_init(elev=elev, azim=azim)

   # Define the faces of the icosahedron using unique_pyramids
    negative_faces = slice(8,18)
    faces = [pyramid[1:] for pyramid in unique_pyramids]
    #faces = [pyramid[1:] for pyramid in unique_pyramids]

    #Adjusting colors based r, if it anterior or posterior
    modified_colors = []
    linewidths = []
    for i, color in enumerate(colors):
        if i in range(negative_faces.start, negative_faces.stop):  # This checks if the index is within negative_faces
            if r > 1:
                alpha = transparency
                linewidth = 1 #Change to 0 for individual subject 
            else:  # Set alpha to 1 if r > 1 and face is in anterior view
                alpha = 0
                linewidth = 1
        else:
            if r > 1:
                alpha = 0 
                linewidth = 1
            else:  # Set alpha to 0 if r > 1 and face is not in anterior view
                alpha = transparency
                linewidth = 1
        modified_colors.append((*color[:3], alpha))
        linewidths.append(linewidth)
    
    poly3d = Poly3DCollection(faces, facecolors=modified_colors, edgecolors='k', linewidths=linewidths)
    ax.add_collection3d(poly3d)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.auto_scale_xyz([-1.05, 0.95], [-0.82, 1.22], [0.76, 2.76])
        
def generate_colors(points_in_pyramid, point_in_pyramid2=None):
    if point_in_pyramid2 is None:
    # normalize the values, adjust colorscale accordingly
        if np.isscalar(points_in_pyramid):
            max_abs = 10
            norm_value = points_in_pyramid / max_abs  # Scale the single value
            color = [plt.cm.Reds(norm_value) if points_in_pyramid >= 0 else plt.cm.Blues(-norm_value)]
            return color  # Return the single color as a list
        else:
            max_abs = np.max(np.abs(points_in_pyramid))
            norm = Normalize(vmin=0, vmax=max_abs)
            norm_values = [norm(point) for point in points_in_pyramid]
            # Red for positive, blue for negative time stroke vs neurotypical
            colors = [plt.cm.Reds(norm(point)) if point >= 0 else plt.cm.Blues(-norm(point)) for point in points_in_pyramid]
    else:
        max_abs = max(max(abs(num) for num in points_in_pyramid), max(abs(num) for num in point_in_pyramid2))
        norm = Normalize(vmin=0, vmax=max_abs)
        norm_values = [norm(point) for point in points_in_pyramid]
        # Red for positive, blue for negative time stroke vs neurotypical
        colors = [plt.cm.Reds(norm(point)) if point >= 0 else plt.cm.Blues(-norm(point)) for point in points_in_pyramid]
        
    return colors


def fixing_bin_center(data, n_bins, center, right):
    x = data[:, :3]  # assuming data is a pandas DataFrame
    #print(x.shape)

    #Previous method does not differentiate the stroke vs neurotypical group bc limited ROM is not highlighted. Instead, find max diff as x-axis
    distances = np.linalg.norm(x, axis=1)  # Create vector of distances to center
    #print(distances)
    filtered_distances = distances[~np.isnan(distances)]
    furthest_distance = np.max(filtered_distances)
    #print('furthest_distance')
    #print(furthest_distance)
    #Define the furthest distance away as arm length and make a cube around the shoulder joint
    mins = np.array([-furthest_distance, -furthest_distance, -furthest_distance])
    ranges = np.array([2*furthest_distance, 2*furthest_distance, 2*furthest_distance])

    N, n_dim = x.shape
    if N <= 2:
        print("X: >2 rows.")
        return
    
    #print(f"\n Sorting ({N} points, {n_dim} dimensions, {n_bins} Bins)...")
    xb = np.full(x.shape, np.nan)  # NaN array like MATLAB NaN*X

    bin_limits = np.zeros((n_bins + 1, n_dim))
    bin_centers = np.zeros((n_bins, n_dim))

    for dim in range(n_dim):
        min_val = mins[dim]
        range_val = ranges[dim]
        #fraction of range
        x_fract_of_range = (x[:, dim] - min_val) / (range_val + np.finfo(float).eps)
        
        # transform to bin space and make integer by flooring to the min value in each bin
        xb[:, dim] = np.floor(n_bins * x_fract_of_range).astype(int)
        
        # Bin limits and centers
        bin_limits[:, dim] = min_val + range_val / n_bins * np.arange(n_bins + 1)
        bin_centers[:, dim] = (bin_limits[:-1, dim] + bin_limits[1:, dim]) / 2
    
    return xb, bin_limits, bin_centers

def linearRegression(fmue_scores, dependentVar, zone, fig, title, color_count):
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#1b9e77', '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#1b9e77', '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d']
    if len(dependentVar) > 10:
        fmue_scores = np.array(fmue_scores)
        dependentVar = np.array(dependentVar)
        regressionValues = LinearRegression().fit(fmue_scores.reshape(-1, 1), dependentVar)
        slope = regressionValues.coef_[0]
        percent_pred_Values = regressionValues.predict(fmue_scores.reshape(-1, 1))
        r2 = r2_score(dependentVar, percent_pred_Values)
        r = spearmanr(fmue_scores, dependentVar)
        p_val = r.pvalue
        print(f"{title} Zone {zone + 1}: Slope={slope}, R^2={r2}, R={r}")
        if p_val <= 0.01:
            fig.add_trace(go.Scatter(x=fmue_scores, y=dependentVar, mode='markers', marker=dict(color=colors[color_count]), name=f'Zone {zone+1}'))
            fig.add_trace(go.Scatter(x=fmue_scores, y=percent_pred_Values, mode='lines', line=dict(color=colors[color_count]), showlegend=False))

def jointAnalysis(df):
    #reference vectors:
    shoulderElbowvectorX = df['Elbow_X'] - df['Shoulder_X']
    shoulderElbowvectorY = df['Elbow_Y'] - df['Shoulder_Y']
    shoulderElbowvectorZ = df['Elbow_Z'] - df['Shoulder_Z']
    shoulderElbowvector = np.column_stack((shoulderElbowvectorX, shoulderElbowvectorY, shoulderElbowvectorZ))
    shoulderElbowvector /= np.linalg.norm(shoulderElbowvector)

    elbowWristvectorX = df['Wrist_X'] - df['Elbow_X']
    elbowWristvectorY = df['Wrist_Y'] - df['Elbow_Y']
    elbowWristvectorZ = df['Wrist_Z'] - df['Elbow_Z']
    elbowWristvector = np.column_stack((elbowWristvectorX, elbowWristvectorY, elbowWristvectorZ))
    elbowWristvector /= np.linalg.norm(elbowWristvector)

    shoulder2shouldervectorX = df['ShoulderRef_X'] - df['ShoulderOther_X']
    shoulder2shouldervectorY = df['ShoulderRef_Y'] - df['ShoulderOther_Y']
    shoulder2shouldervectorZ = df['ShoulderRef_Z'] - df['ShoulderOther_Z']
    shoulder2shouldervector = np.column_stack((shoulder2shouldervectorX, shoulder2shouldervectorY, shoulder2shouldervectorZ))
    shoulder2shouldervector /= np.linalg.norm(shoulder2shouldervector)

    spineVectorX = df['SpineShoulder_X'] - df['SpineBase_X']
    spineVectorY = df['SpineShoulder_Y'] - df['SpineBase_Y']
    spineVectorZ = df['SpineShoulder_Z'] - df['SpineBase_Z']
    spineVector = np.column_stack((spineVectorX, spineVectorY, spineVectorZ))
    spineVector /= np.linalg.norm(spineVector)

    #rotation matrices
    rotation_matrix = np.vstack([shoulder2shouldervector, spineVector, np.cross(shoulder2shouldervector, spineVector)]).T
    
    #calculate Euler angles (ZYX order for intrinsic rotations)
    rotation = R.from_matrix(rotation_matrix)
    euler_angles = rotation.as_euler('zyx', degrees=True)
    
    #print("Shoulder Flexion Angle (Euler ZYX):", euler_angles)

    #Shoulder Flexion
    flexion_angle = euler_angles[1]  # Y-axis rotation (Flexion)

    #Shoulder Abduction
    abduction_angle = euler_angles[2] # Z-axis rotation (Abduction)

    #Elbow Flexion -  just use 3D points
    ElbowAngles = np.zeros(df.shape[0])
    for row in range(df.shape[0]):
        dot_Elbow = np.dot(shoulderElbowvector[row], elbowWristvector[row])
        magnitude_ElbShoul = np.linalg.norm(elbowShouldervector[row])
        magnitude_ElbWrist = np.linalg.norm(elbowWristvector[row])
        cos_theta = dot_Elbow / (magnitude_ElbShoul * magnitude_ElbWrist)
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        ElbowAngles[row] = np.degrees(np.arccos(cos_theta))

import numpy as np

def calc_jointangles(df):
    # Initialize lists to store vectors and angles
    shoulderElbowvectorX = df['Elbow_X'] - df['Shoulder_X']
    shoulderElbowvectorY = df['Elbow_Y'] - df['Shoulder_Y']
    shoulderElbowvectorZ = df['Elbow_Z'] - df['Shoulder_Z']
    shoulderElbowvector = np.column_stack((shoulderElbowvectorX, shoulderElbowvectorY, shoulderElbowvectorZ))
    shoulderElbowvector /= np.linalg.norm(shoulderElbowvector)

    elbowWristvectorX = df['Wrist_X'] - df['Elbow_X']
    elbowWristvectorY = df['Wrist_Y'] - df['Elbow_Y']
    elbowWristvectorZ = df['Wrist_Z'] - df['Elbow_Z']
    elbowWristvector = np.column_stack((elbowWristvectorX, elbowWristvectorY, elbowWristvectorZ))
    elbowWristvector /= np.linalg.norm(elbowWristvector)

    wristHandvectorX = df['Hand_X'] - df['Wrist_X']
    wristHandvectorY = df['Hand_Y'] - df['Wrist_Y']
    wristHandvectorZ = df['Hand_Z'] - df['Wrist_Z']
    wristHandvector = np.column_stack((wristHandvectorX, wristHandvectorY, wristHandvectorZ))
    wristHandvector /= np.linalg.norm(wristHandvector)

    #transverse vector - used for calculating flexion
    shoulder2shouldervectorX = df['ShoulderRef_X'] - df['ShoulderOther_X']
    shoulder2shouldervectorY = df['ShoulderRef_Y'] - df['ShoulderOther_Y']
    shoulder2shouldervectorZ = df['ShoulderRef_Z'] - df['ShoulderOther_Z']
    shoulder2shouldervector = np.column_stack((shoulder2shouldervectorX, shoulder2shouldervectorY, shoulder2shouldervectorZ))
    shoulder2shouldervector /= np.linalg.norm(shoulder2shouldervector)

    #vertical vector - used for calculating abduction
    hip2shouldervectorX = df['ShoulderRef_X'] - df['Hip_X']
    hip2shouldervectorY = df['ShoulderRef_Y'] - df['Hip_Y']
    hip2shouldervectorZ = df['ShoulderRef_Z'] - df['Hip_Z']
    hip2shouldervector = np.column_stack((hip2shouldervectorX, hip2shouldervectorY, hip2shouldervectorZ))
    hip2shouldervector /= np.linalg.norm(hip2shouldervector)
    
    # Initialize an array to store joint angles
    joint_angles = np.zeros((len(shoulderElbowvectorX), 4))

    # Shoulder abduction/adduction - angle between hipshoulder and elbowshoulder
    for i in range(len(shoulderElbowvectorX)):
        cross_product = np.cross(hip2shouldervector[i], shoulderElbowvector[i])
        dot_product = np.dot(hip2shouldervector[i], shoulderElbowvector[i])
        joint_angles[i, 0] = np.degrees(np.arctan2(np.linalg.norm(cross_product), dot_product))

    # Shoulder flexion/extension - angle between spineshoulder and elbowshoulder
        cross_product = np.cross(shoulder2shouldervector[i], shoulderElbowvector[i])
        dot_product = np.dot(shoulder2shouldervector[i], shoulderElbowvector[i])
        joint_angles[i, 1] = np.degrees(np.arctan2(np.linalg.norm(cross_product), dot_product))

    # Elbow flexion/extension - angle between elbowwrist and elbowshoulder
        cross_product = np.cross(shoulderElbowvector[i], elbowWristvector[i])
        dot_product = np.dot(shoulderElbowvector[i], elbowWristvector[i])
        joint_angles[i, 2] = np.degrees(np.arctan2(np.linalg.norm(cross_product), dot_product))

    #Wrist flexion/extension - angle between wristHand and elbowWrist
        cross_product = np.cross(elbowWristvector[i], wristHandvector[i])
        dot_product = np.dot(elbowWristvector[i], wristHandvector[i])
        joint_angles[i, 3] = np.degrees(np.arctan2(np.linalg.norm(cross_product), dot_product))

    #title = "Shoulder Flexion v. Elbow Flexion"
    #fig = px.scatter(x=joint_angles[:, 1], y=joint_angles[:, 2])
    #fig.update_layout(
    #    title=title,
    #    xaxis=dict(title='Shoulder Flexion', range=[0, 180]),
    #    yaxis=dict(title='Elbow Flexion', range = [0, 180]),
    #    width=600,
    #    height=600,
    #    plot_bgcolor='white',
    #    showlegend=False,
    #    margin=dict(l=50, r=50, b=50, t=50),
    #)
    #fig.show()

    #title =  "Shoulder Abduction v Elbow Flexion"
    #fig = px.scatter(x=joint_angles[:, 0], y=joint_angles[:, 2])
    #fig.update_layout(
    #    title=title,
    #    xaxis=dict(title='Shoulder Abduction', range = [0, 180]),
    #    yaxis=dict(title='Elbow Flexion', range = [0, 180]),
    #    width=600,
    #    height=600,
    #    plot_bgcolor='white',
    #    showlegend=False,
    #    margin=dict(l=50, r=50, b=50, t=50),
    #)
    #fig.show()

    #title =  "Shoulder Flexion v Shoulder Abduction"
    #fig = px.scatter(x=joint_angles[:, 1], y=joint_angles[:, 0])
    #fig.update_layout(
    #    title=title,
    #    xaxis=dict(title='Shoulder Flexion', range = [0, 180]),
    #    yaxis=dict(title='Shoulder Abduction', range = [0, 180]),
    #    width=600,
    #    height=600,
    #    plot_bgcolor='white',
    #    showlegend=False,
    #    margin=dict(l=50, r=50, b=50, t=50),
    #)
    #fig.show()

    #title = "Elbow Flexion v Wrist Flexion"
    #fig = px.scatter(x=joint_angles[:, 2], y=joint_angles[:, 3])
    #fig.update_layout(
    #    title=title,
    #    xaxis=dict(title='Elbow Flexion'),
    #    yaxis=dict(title='Wrist Flexion'),
    #    width=600,
    #    height=600,
    #    plot_bgcolor='white',
    #    showlegend=False,
    #    margin=dict(l=50, r=50, b=50, t=50),
    #)
    #fig.show()

    #3D plot tests: shoulder abduction v. elbow flexion v. shoulder flexion
    fig = go.Figure(data=[
        go.Scatter3d(
            x=joint_angles[:, 0],
            y=joint_angles[:, 2],
            z=joint_angles[:, 1], 
            mode='markers',
            marker=dict(size=2, color='blue'),
        )
    ])
    
    fig.update_layout(
        title="3D scatter of the shoulder and elbow angles",
        scene=dict(
            xaxis=dict(title='Shoulder Abduction'),
            yaxis=dict(title='Elbow Flexion'),
            zaxis=dict(title='Shoulder Flexion'),
        ),
        width=800,
        height=800,
    )
    
    fig.show()

    return joint_angles


def jointAngleCalculations(df, right):
    #Merge points - tired of all this right and left stuff

    #initialize jointAngles_summary
    JointAngle_summary = [[] for _ in range(9)]
    JointCorrelation_summary = [[] for _ in range(3)]
    #check angles shoulder -  3 points - shoulder_right, hip_right, elbow_right
    ShoulderAngles = np.zeros(df.shape[0])
    shoulderHipvectorX = df['Hip_X'] -  df['Shoulder_X']
    shoulderHipvectorY = df['Hip_Y'] -  df['Shoulder_Y']
    shoulderHipvectorZ = df['Hip_Z'] -  df['Shoulder_Z']
    shoulderHipvector = np.column_stack((shoulderHipvectorX, shoulderHipvectorY, shoulderHipvectorZ))
    
    shoulderElbowvectorX = df['Elbow_X'] - df['Shoulder_X']
    shoulderElbowvectorY = df['Elbow_Y'] - df['Shoulder_Y']
    shoulderElbowvectorZ = df['Elbow_Z'] - df['Shoulder_Z']
    shoulderElbowvector = np.column_stack((shoulderElbowvectorX, shoulderElbowvectorY, shoulderElbowvectorZ))
    
    for row in range(df.shape[0]):
        dot_Shoulder = np.dot(shoulderHipvector[row], shoulderElbowvector[row])
        magnitude_SholHip = np.linalg.norm(shoulderHipvector[row])
        magnitude_SholElb = np.linalg.norm(shoulderElbowvector[row])
        cos_theta = dot_Shoulder / (magnitude_SholHip * magnitude_SholElb)
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        ShoulderAngles[row] = np.degrees(np.arccos(cos_theta))

    median = np.median(ShoulderAngles)
    mode = stats.mode(ShoulderAngles)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(ShoulderAngles)  # ptp = peak to peak (max - min)
    variance = np.var(ShoulderAngles, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(ShoulderAngles, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(ShoulderAngles)
    skewness = stats.skew(ShoulderAngles)
    kurt = stats.kurtosis(ShoulderAngles)
    #print(f'Shoulder Angle - mean: {mean}, median: {median}, mode: {mode}, range: {data_range}, variance: {variance}, std_dev: {std_dev}, iqr: {iqr}, skewness: {skewness}, kurtosis: {kurt}')
    ShoulderAngleSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    #plt.hist(ShoulderAngles, bins='auto', edgecolor='black')
    #plt.title('Shoulder Histogram of Angles (degrees)')
    #plt.xlabel('Angle (degrees)')
    #plt.ylabel('Frequency')
    #plt.show()
    JointAngle_summary[0] = ShoulderAngleSummary
    #print(f'Range Shoulder Angle: {[np.min(ShoulderAngles), np.max(ShoulderAngles)]}')
        
    #Check angles elbow - 3 points - elbow_right, shoulder_right, wrist_right
    ElbowAngles = np.zeros(df.shape[0])
    elbowShouldervectorX = df['Shoulder_X'] - df['Elbow_X']
    elbowShouldervectorY = df['Shoulder_Y'] - df['Elbow_Y']
    elbowShouldervectorZ = df['Shoulder_Z'] - df['Elbow_Z']
    elbowShouldervector = np.column_stack((elbowShouldervectorX, elbowShouldervectorY, elbowShouldervectorZ))
    
    elbowWristvectorX = df['Wrist_X'] - df['Elbow_X']
    elbowWristvectorY = df['Wrist_Y'] - df['Elbow_Y']
    elbowWristvectorZ = df['Wrist_Z'] - df['Elbow_Z']
    elbowWristvector = np.column_stack((elbowWristvectorX, elbowWristvectorY, elbowWristvectorZ))
    
    for row in range(df.shape[0]):
        dot_Elbow = np.dot(elbowShouldervector[row], elbowWristvector[row])
        magnitude_ElbShoul = np.linalg.norm(elbowShouldervector[row])
        magnitude_ElbWrist = np.linalg.norm(elbowWristvector[row])
        cos_theta = dot_Elbow / (magnitude_ElbShoul * magnitude_ElbWrist)
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        ElbowAngles[row] = np.degrees(np.arccos(cos_theta))
    
    median = np.median(ElbowAngles)
    mode = stats.mode(ElbowAngles)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(ElbowAngles)  # ptp = peak to peak (max - min)
    variance = np.var(ElbowAngles, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(ElbowAngles, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(ElbowAngles)
    skewness = stats.skew(ElbowAngles)
    kurt = stats.kurtosis(ElbowAngles)
    #print(f'Elbow Angle - mean: {mean}, median: {median}, mode: {mode}, range: {data_range}, variance: {variance}, std_dev: {std_dev}, iqr: {iqr}, skewness: {skewness}, kurtosis: {kurt}')
    ElbowAngleSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    #plt.hist(ElbowAngles, bins='auto', edgecolor='black')
    #plt.title('Elbow Histogram of Angles (degrees)')
    #plt.xlabel('Angle (degrees)')
    #plt.ylabel('Frequency')
    #plt.show()
    JointAngle_summary[1] = ElbowAngleSummary
    #print(f'Range Elbow Angle: {[np.min(ElbowAngles), np.max(ElbowAngles)]}')
    
    #check angles wrist -  3points - wrist_right, elbow_right, hand_right
    WristAngles = np.zeros(df.shape[0])
    wristElbowvectorX = df['Elbow_X'] - df['Wrist_X']
    wristElbowvectorY = df['Elbow_Y'] - df['Wrist_Y']
    wristElbowvectorZ = df['Elbow_Z'] - df['Wrist_Z']
    wristElbowvector = np.column_stack((wristElbowvectorX, wristElbowvectorY, wristElbowvectorZ))
    
    wristHandvectorX = df['Hand_X'] - df['Wrist_X']
    wristHandvectorY = df['Hand_Y'] - df['Wrist_Y']
    wristHandvectorZ = df['Hand_Z'] - df['Wrist_Z']
    wristHandvector = np.column_stack((wristHandvectorX, wristHandvectorY, wristHandvectorZ))
    
    for row in range(df.shape[0]):
        dot_Wrist = np.dot(wristElbowvector[row], wristHandvector[row])
        magnitude_WriElb = np.linalg.norm(wristElbowvector[row])
        magnitude_WriHan = np.linalg.norm(wristHandvector[row])
        cos_theta = dot_Wrist / (magnitude_WriElb * magnitude_WriHan)
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        WristAngles[row] = np.degrees(np.arccos(cos_theta))
    
    median = np.median(WristAngles)
    mode = stats.mode(WristAngles)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(WristAngles)  # ptp = peak to peak (max - min)
    variance = np.var(WristAngles, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(WristAngles, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(WristAngles)
    skewness = stats.skew(WristAngles)
    kurt = stats.kurtosis(WristAngles)
    #print(f'Wrist Angle - mean: {mean}, median: {median}, mode: {mode}, range: {data_range}, variance: {variance}, std_dev: {std_dev}, iqr: {iqr}, skewness: {skewness}, kurtosis: {kurt}')
    WristAngleSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    #plt.hist(WristAngles, bins='auto', edgecolor='black')
    #plt.title('Wrist Histogram of Angles (degrees)')
    #plt.xlabel('Angle (degrees)')
    #plt.ylabel('Frequency')
    #plt.show()
    JointAngle_summary[2] = WristAngleSummary
    #print(f'Range Wrist Angle: {[np.min(WristAngles), np.max(WristAngles)]}')

    #Jointangle correlations & velocity and acceleration calculations
    fps=30
    #Find the angular velocities and accelerations
    Vel_ShoulderAngle = np.diff(ShoulderAngles) * fps
    Vel_ElbowAngle = np.diff(ElbowAngles) * fps
    Vel_WristAngle = np.diff(WristAngles) * fps
    Acc_ShoulderAngle = np.diff(Vel_ShoulderAngle) * fps
    Acc_ElbowAngle = np.diff(Vel_ElbowAngle) * fps
    Acc_WristAngle = np.diff(Vel_WristAngle) * fps

    #Do the correlation math
    r_shoulElb, p_val = pearsonr(ShoulderAngles, ElbowAngles)
    r_elbWri, p_val = pearsonr(ElbowAngles, WristAngles)
    r_shoulWri, p_val = pearsonr(ShoulderAngles, WristAngles)

    r_Vel_shoulElb, p_val = pearsonr(Vel_ShoulderAngle, Vel_ElbowAngle)
    r_Vel_elbWri, p_val = pearsonr(Vel_ElbowAngle, Vel_WristAngle)
    r_Vel_shoulWri, p_val = pearsonr(Vel_ShoulderAngle, Vel_WristAngle)

    r_Acc_shoulElb, p_val = pearsonr(Acc_ShoulderAngle, Acc_ElbowAngle)
    r_Acc_elbWri, p_val = pearsonr(Acc_ElbowAngle, Acc_WristAngle)
    r_Acc_shoulWri, p_val = pearsonr(Acc_ShoulderAngle, Acc_WristAngle)

    Displacement_summary = [r_shoulElb, r_elbWri, r_shoulWri]
    Velocity_summary = [r_Vel_shoulElb, r_Vel_elbWri, r_Vel_shoulWri]
    Acceleration_summary = [r_Acc_shoulElb, r_Acc_elbWri, r_Acc_shoulWri]
    JointCorrelation_summary[0] = Displacement_summary
    JointCorrelation_summary[1] = Velocity_summary
    JointCorrelation_summary[2] = Acceleration_summary

    #summary details of angular velocity - Shoulder and Elbow
    median = np.median(Vel_ShoulderAngle)
    mode = stats.mode(Vel_ShoulderAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Vel_ShoulderAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Vel_ShoulderAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Vel_ShoulderAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Vel_ShoulderAngle)
    skewness = stats.skew(Vel_ShoulderAngle)
    kurt = stats.kurtosis(Vel_ShoulderAngle)
    AngVel_ShoulSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[3] = AngVel_ShoulSummary

    median = np.median(Vel_ElbowAngle)
    mode = stats.mode(Vel_ElbowAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Vel_ElbowAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Vel_ElbowAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Vel_ElbowAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Vel_ElbowAngle)
    skewness = stats.skew(Vel_ElbowAngle)
    kurt = stats.kurtosis(Vel_ElbowAngle)
    AngVel_ElbowSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[4] = AngVel_ElbowSummary

    median = np.median(Vel_WristAngle)
    mode = stats.mode(Vel_WristAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Vel_WristAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Vel_WristAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Vel_WristAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Vel_WristAngle)
    skewness = stats.skew(Vel_WristAngle)
    kurt = stats.kurtosis(Vel_WristAngle)
    AngVel_WristSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[5] = AngVel_WristSummary

    #Angular Acceleration
    median = np.median(Acc_ShoulderAngle)
    mode = stats.mode(Acc_ShoulderAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Acc_ShoulderAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Acc_ShoulderAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Acc_ShoulderAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Acc_ShoulderAngle)
    skewness = stats.skew(Acc_ShoulderAngle)
    kurt = stats.kurtosis(Acc_ShoulderAngle)
    AngAcc_ShoulSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[6] = AngAcc_ShoulSummary

    median = np.median(Acc_ElbowAngle)
    mode = stats.mode(Acc_ElbowAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Acc_ElbowAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Acc_ElbowAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Acc_ElbowAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Acc_ElbowAngle)
    skewness = stats.skew(Acc_ElbowAngle)
    kurt = stats.kurtosis(Acc_ElbowAngle)
    AngAcc_ElbowSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[7] = AngAcc_ElbowSummary

    median = np.median(Acc_WristAngle)
    mode = stats.mode(Acc_WristAngle)[0][0]  # mode returns a ModeResult object
    data_range = np.ptp(Acc_WristAngle)  # ptp = peak to peak (max - min)
    variance = np.var(Acc_WristAngle, ddof=1)  # ddof=1 for sample variance
    std_dev = np.std(Acc_WristAngle, ddof=1)  # ddof=1 for sample standard deviation
    iqr = stats.iqr(Acc_WristAngle)
    skewness = stats.skew(Acc_WristAngle)
    kurt = stats.kurtosis(Acc_WristAngle)
    AngAcc_WristSummary = [median, mode, data_range, variance, std_dev, iqr, skewness, kurt]
    JointAngle_summary[8] = AngAcc_WristSummary

        #ElbowAngles = filterAngles(ElbowAngles)
        #plt.hist(ElbowAngles, bins=55, edgecolor='black')
        #plt.xlim(0, 180)
        #plt.title('Elbow Angles Distribution')
        #plt.xlabel('Deg')
        #plt.ylabel('Frequency')
        #plt.show()

        #Vel_ElbowAngle = filterAngles(Vel_ElbowAngle)
        #plt.hist(Vel_ElbowAngle, bins=55, edgecolor='black')
        #plt.title('Elbow Anglular Velocity Distribution')
        #plt.xlabel('Deg/s')
        #plt.ylabel('Frequency')
        #plt.show()
        
        #Acc_ElbowAngle = filterAngles(Acc_ElbowAngle)
        #plt.hist(Acc_ElbowAngle, bins=55, edgecolor='black')
        #plt.title('Elbow Anglular Acceleration Distribution')
        #plt.xlabel('Deg/s^2')
        #plt.ylabel('Frequency')
        #plt.show()
    
    return JointCorrelation_summary, JointAngle_summary, ElbowAngles, ShoulderAngles, WristAngles

def distCalculations(zone_values):
    summaryList = []
    if len(zone_values) > 4:
        zone_values = np.round(zone_values, decimals=4)
        median = np.median(zone_values)
        mode = stats.mode(zone_values)[0][0]  # mode returns a ModeResult object
        data_range = np.ptp(zone_values)  # ptp = peak to peak (max - min)
        variance = np.var(zone_values, ddof=1)  # ddof=1 for sample variance
        std_dev = np.std(zone_values, ddof=1)  # ddof=1 for sample standard deviation
        iqr = stats.iqr(zone_values)
        skewness = skew(zone_values)
            #Leptokurtosis
        kurt = kurtosis(zone_values, fisher=False)
        summaryList = [median, mode, data_range, std_dev, iqr, skewness, kurt, len(zone_values)]
    return summaryList

def shape_modeling(df, right, subject, jointAngle_list, stats_summary, stats_icosahedron, stats_rectilinear, stats_vel_icos, stats_acc_icos, stats_forceavg_icosahedron, stats_forceavg_rectilinear, stats_jointAngavg_icosahedron, stats_synergy_icosahedron, stats_mvt_icosahedron, cumData):
    if right == 1:
        shoulder_cols = ['ShoulderRight_X', 'ShoulderRight_Y', 'ShoulderRight_Z']
        wrist_cols = ['WristRight_X', 'WristRight_Y', 'WristRight_Z']
    else:
        shoulder_cols = ['ShoulderLeft_X', 'ShoulderLeft_Y', 'ShoulderLeft_Z']
        wrist_cols = ['WristLeft_X', 'WristLeft_Y', 'WristLeft_Z']
    
    
    center = df[shoulder_cols].loc[:100, :].mean()
    center.astype(np.int64)
    df = df.drop(df.index[-1])
    if right == 1:
        df['Wrist_X'] = df['WristRight_X'] - center['ShoulderRight_X']
        df['Wrist_Y'] = df['WristRight_Y'] - center['ShoulderRight_Y']
        df['Wrist_Z'] = df['WristRight_Z'] - center['ShoulderRight_Z']
        df['Hip_X'] = df['HipRight_X'] - center['ShoulderRight_X']
        df['Hip_Y'] = df['HipRight_Y'] - center['ShoulderRight_Y']
        df['Hip_Z'] = df['HipRight_Z'] - center['ShoulderRight_Z']
        df['Elbow_X'] = df['ElbowRight_X'] - center['ShoulderRight_X']
        df['Elbow_Y'] = df['ElbowRight_Y'] - center['ShoulderRight_Y']
        df['Elbow_Z'] = df['ElbowRight_Z'] - center['ShoulderRight_Z']
        df['Hand_X'] = df['HandRight_X'] - center['ShoulderRight_X']
        df['Hand_Y'] = df['HandRight_Y'] - center['ShoulderRight_Y']
        df['Hand_Z'] = df['HandRight_Z'] - center['ShoulderRight_Z']
        df['Shoulder_X'] = df['ShoulderRight_X'] - center['ShoulderRight_X']
        df['Shoulder_Y'] = df['ShoulderRight_Y'] - center['ShoulderRight_Y']
        df['Shoulder_Z'] = df['ShoulderRight_Z'] - center['ShoulderRight_Z']
        df['Thumb_X'] = df['ThumbRight_X'] - center['ShoulderRight_X']
        df['Thumb_Y'] = df['ThumbRight_Y'] - center['ShoulderRight_Y']
        df['Thumb_Z'] = df['ThumbRight_Z'] - center['ShoulderRight_Z']
        df['ShoulderRef_X'] = df['Shoulder_X']
        df['ShoulderRef_Y'] = df['Shoulder_Y']
        df['ShoulderRef_Z'] = df['Shoulder_Z']
        df['ShoulderOther_X'] = df['ShoulderLeft_X']
        df['ShoulderOther_Y'] = df['ShoulderLeft_Y']
        df['ShoulderOther_Z'] = df['ShoulderLeft_Z']
        center['ShoulderRight_X'] -= center['ShoulderRight_X']
        center['ShoulderRight_Y'] -= center['ShoulderRight_Y']
        center['ShoulderRight_Z'] -= center['ShoulderRight_Z']
    else:
        df['Wrist_X'] = -(df['WristLeft_X'] - center['ShoulderLeft_X'])
        df['Wrist_Y'] = df['WristLeft_Y'] - center['ShoulderLeft_Y']
        df['Wrist_Z'] = df['WristLeft_Z'] - center['ShoulderLeft_Z']
        df['Hip_X'] = -(df['HipLeft_X'] - center['ShoulderLeft_X'])
        df['Hip_Y'] = df['HipLeft_Y'] - center['ShoulderLeft_Y']
        df['Hip_Z'] = df['HipLeft_Z'] - center['ShoulderLeft_Z']
        df['Elbow_X'] = -(df['ElbowLeft_X'] - center['ShoulderLeft_X'])
        df['Elbow_Y'] = df['ElbowLeft_Y'] - center['ShoulderLeft_Y']
        df['Elbow_Z'] = df['ElbowLeft_Z'] - center['ShoulderLeft_Z']
        df['Hand_X'] = -(df['HandLeft_X'] - center['ShoulderLeft_X'])
        df['Hand_Y'] = df['HandLeft_Y'] - center['ShoulderLeft_Y']
        df['Hand_Z'] = df['HandLeft_Z'] - center['ShoulderLeft_Z']
        df['Shoulder_X'] = -(df['ShoulderLeft_X'] - center['ShoulderLeft_X'])
        df['Shoulder_Y'] = df['ShoulderLeft_Y'] - center['ShoulderLeft_Y']
        df['Shoulder_Z'] = df['ShoulderLeft_Z'] - center['ShoulderLeft_Z']
        df['Thumb_X'] = -(df['ThumbLeft_X'] - center['ShoulderLeft_X'])
        df['Thumb_Y'] = df['ThumbLeft_Y'] - center['ShoulderLeft_Y']
        df['Thumb_Z'] = df['ThumbLeft_Z'] - center['ShoulderLeft_Z']
        df['ShoulderRef_X'] = df['Shoulder_X']
        df['ShoulderRef_Y'] = df['Shoulder_Y']
        df['ShoulderRef_Z'] = df['Shoulder_Z']
        df['ShoulderOther_X'] = df['ShoulderRight_X']
        df['ShoulderOther_Y'] = df['ShoulderRight_Y']
        df['ShoulderOther_Z'] = df['ShoulderRight_Z']
        center['ShoulderLeft_X'] -= center['ShoulderLeft_X']
        center['ShoulderLeft_Y'] -= center['ShoulderLeft_Y']
        center['ShoulderLeft_Z'] -= center['ShoulderLeft_Z']
        #call filter95 function to cut >95% percentile distances from shoulder

    fs = 30
    trim_for_abstract = fs * 120 #trim the first 2 minutes of the data
    df = df.loc[:trim_for_abstract, :]
    
    #print(f'Filter95 Check {df.shape}')
    df = filterPositionData(df, center, right, subject)
    #df = df_filtered 
    
    #fig2 = px.scatter_3d(df, x="Wrist_X", y="Wrist_Y", z="Wrist_Z")
    #fig2.update_layout(height = 500, width = 500)
    #fig2.update_traces(marker = dict(size = 2))
    #fig2.show()
    
    #fig3 = px.scatter_3d(df_filtered, x="Wrist_X", y="Wrist_Y", z="Wrist_Z")
    #fig3.update_layout(height = 500, width = 500)
    #fig3.update_traces(marker = dict(size = 2))
    
    #fig3.add_trace(go.Scatter3d(
    #    x=[0],  # Origin x
    #    y=[0],  # Origin y
    #    z=[0],
    #    mode='markers',  # Show marker only
    #    marker=dict(color='red', size=5),  # Red color and size of the marker
    #    name='Origin'
    #))
    
    #fig3.show()
    
    #testing distribution as spherical - Calculate radius for Icosahedron vertices
    vectorX = df['Wrist_X'] 
    vectorY = df['Wrist_Y']
    vectorZ = df['Wrist_Z'] 
    vectors=np.column_stack((vectorX[:-1], vectorY[:-1], vectorZ[:-1]))
    vectors_normalized = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis]
    azimuthal_angles = np.arctan2(vectors_normalized[:, 1], vectors_normalized[:, 0])
    polar_angles = np.arctan2(vectors_normalized[:, 2], np.sqrt(vectors_normalized[:, 0] ** 2 + vectors_normalized[:, 1] ** 2))
    
    #find the radius of the spherical distribution
    magnitudes = np.linalg.norm(vectors, axis=1)
    largest_magnitude = np.max(magnitudes)
    avg_magnitude = np.mean(magnitudes)
    med_magnitude = np.median(magnitudes)
    radius = largest_magnitude
    #print(f'radius: {radius}')
    
    # golden ratio
    phi = (1 + np.sqrt(5)) / 2
    
    # vertices of a unit icosahedron
    icosahedron_vertices = np.array([
        [0, 1, phi],
        [0, -1, phi],
        [0, 1, -phi],
        [0, -1, -phi],
        [1, phi, 0],
        [-1, phi, 0],
        [1, -phi, 0],
        [-1, -phi, 0],
        [phi, 0, 1],
        [-phi, 0, 1],
        [phi, 0, -1],
        [-phi, 0, -1]
    ])
    
    # scaling by the largest magnitude of vector_normalized (from prev calc)
    factorNorm = radius / phi
    icosahedron_vertices *= factorNorm
    
    #fig2.add_trace(go.Scatter3d(
    #    x=icosahedron_vertices[:, 0],
    #    y=icosahedron_vertices[:, 1],
    #    z=icosahedron_vertices[:, 2],
    #    mode='markers',
    #    marker=dict(size=4, color='black')
    #))
    
    # show the figure
    #fig2.show()

    #test 1: find correct unique_pyramids
    # Assuming icosahedron_vertices and center are defined
    center = np.array([0, 0, 0])
    distances = cdist(icosahedron_vertices, icosahedron_vertices)

    # Find the 5 nearest neighbors for each vertex (excluding itself)
    # argsort sorts distances, so [:, 1:6] skips the first column (distance to itself) and selects the next 5 nearest neighbors.
    nearest_neighbors_indices = np.argsort(distances, axis=1)[:, 1:6]
    # Sort nearest neighbors indices for consistent processing
    nearest_neighbors_indices = np.sort(nearest_neighbors_indices, axis=1)
    
    unique_pyramids_set = set()
    for vertex_idx, neighbors_idx in enumerate(nearest_neighbors_indices):
        vertex = icosahedron_vertices[vertex_idx]
        neighbors = icosahedron_vertices[neighbors_idx]
    
        # Generate combinations of neighbors
        neighbor_combinations = itertools.combinations(neighbors_idx, 2)
    
        for combination_idx in neighbor_combinations:
            combination = icosahedron_vertices[np.array([vertex_idx, *combination_idx])]
            combination = combination[np.lexsort(combination.T)]  # Sort vertices of the pyramid
    
            # Calculate distances
            dist_01 = np.linalg.norm(combination[0] - combination[1])
            dist_12 = np.linalg.norm(combination[1] - combination[2])
            dist_20 = np.linalg.norm(combination[2] - combination[0])
            
            # Check for equilateral triangle
            mean_dist = np.mean([dist_01, dist_12, dist_20])
            if np.allclose([dist_01, dist_12, dist_20], mean_dist, atol=1e-6):
                # Add sorted combination to the set with the center
                sorted_indices = tuple(sorted([vertex_idx, *combination_idx]))
                unique_pyramids_set.add(sorted_indices)
    
    # Convert back to list of pyramids and prepend the center point
    unique_pyramids = []
    for indices in unique_pyramids_set:
        pyramid_vertices = icosahedron_vertices[list(indices)]
        pyramid_with_center = np.vstack([center, pyramid_vertices])  # Add center as the apex
        unique_pyramids.append(pyramid_with_center)
    
    # Print the resulting pyramids
    #print("Unique Pyramids:")
    #for pyramid in unique_pyramids:
        #print(pyramid)


    wristMotionX = df['Wrist_X']
    wristMotionY = df['Wrist_Y']
    wristMotionZ = df['Wrist_Z']
    wristMotionOnly=np.column_stack((wristMotionX, wristMotionY, wristMotionZ))

    #get mass from clinical files spreadsheet
    if subject[0] == 's':
        heightWeight_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase0_Heightweight_DATA_LABELS_2024_06_11_1650.csv')
    else:
        heightWeight_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase1-Heightweight_DATA_LABELS_2024_06_11_1654.csv')

    subject_Height = heightWeight_df[heightWeight_df['Record ID'].str.lower() == subject.lower()]['Height (inches)'] * 0.0254
    subject_Height = subject_Height.values[0]
    head_position = np.mean(df['Head_Y'])
    foot_position = np.mean(df['FootLeft_Y'])
    estimated_Height = head_position - foot_position
    #Call on heightWeight_df to get the weights and conver to mass (kg)
    subject_Mass = heightWeight_df[heightWeight_df['Record ID'].str.lower() == subject.lower()]['Weight (lbs)'] * 0.454 #find weight and convert to mass by dividing gravity
    subject_Mass = subject_Mass.values[0]
    Arm_mass = 0.05 * subject_Mass
    #Calculate velocity, acceleration, force (for now - acceleration enough
    vel = np.diff(wristMotionOnly, axis=0) * fs
    #print(f'Velocity {vel.shape}')
    accel = np.diff(vel, axis=0) * fs
    #print(f'Accel {accel.shape}')
    jerk = np.diff(accel, axis=0) * fs
    #print(f'Jerk {jerk.shape}')
    forces = Arm_mass * np.linalg.norm(accel, axis=1) #for now, its just accel magnitude
    veloc = np.linalg.norm(vel, axis=1) #magnitude velocity
    accel = np.linalg.norm(accel, axis=1) #magnitude acceleration
    jerk = np.linalg.norm(jerk, axis=1) #magnitude jerk

    #plot movement
    #make movingaverage filter
    window=5
    samp_rate = 30
    vel_Xma = np.convolve(vel[:,0], np.ones(window)/window, mode='valid')
    vel_Yma = np.convolve(vel[:,1], np.ones(window)/window, mode='valid')
    vel_Zma = np.convolve(vel[:,2], np.ones(window)/window, mode='valid')
    vel_ma = np.convolve(veloc, np.ones(window)/window, mode='valid')
    acc_ma = np.convolve(accel, np.ones(window)/window, mode='valid')
    #filter jerk
    cutoff = 10
    jerk_filtered = butter_lowpass_filter(jerk)
    jerk_ma = np.convolve(jerk, np.ones(window)/window, mode='valid')
    #ma_vel_line = go.Scatter(x=np.arange(len(vel_ma)) + window -1, y=vel_ma, mode='lines', name='Velocity')
    ma_acc_line = go.Scatter(x=np.arange(len(acc_ma))+window-1, y=acc_ma, mode='lines', name='Acceleration')
    ma_jerk_line = go.Scatter(x=np.arange(len(jerk_ma))+window-1, y=jerk_ma, mode='lines', name='Jerk')
    #identify movements
    trough_indices, _ = find_peaks(-jerk_ma, distance=window, prominence=600)  #do i need to adjust this index based on smookthening??
    adj_trough_indices = trough_indices + (window - 1)
    #print(f'Troughs: {len(adj_trough_indices)}')
    #print(adj_trough_indices)
    #figvel = go.Figure()
    #figvel.add_trace(go.Scatter(x=np.arange(len(vel_Xma)) + window -1, y=vel_Xma, mode='lines', name='Vel X', marker=dict(color='red')))
    #figvel.add_trace(go.Scatter(x=np.arange(len(vel_Yma)) + window -1, y=vel_Yma, mode='lines', name='Vel Y', marker=dict(color='blue')))
    #figvel.add_trace(go.Scatter(x=np.arange(len(vel_Zma)) + window -1, y=vel_Zma, mode='lines', name='Vel Z', marker=dict(color='green')))
    #figvel.add_trace(go.Scatter(x=np.arange(len(vel_ma)) + window -1, y=vel_ma, mode='lines', name='Vel mag', marker=dict(color='black')))
    ##figacc = go.Figure(data=[ma_acc_line])
    #figjerk = go.Figure(data=[ma_jerk_line])
    #figjerk.add_trace(go.Scatter(x=adj_trough_indices, y=jerk_ma[trough_indices], mode='markers', marker=dict(symbol='star',size=5, color='red')))
    #layout
    #figvel.update_layout(title='Velocity', xaxis_title='frames', yaxis_title='m/s')
    #figacc.update_layout(title='Acceleration', xaxis_title='frames', yaxis_title='m/(s^2)')
    #figjerk.update_layout(title='Jerk', xaxis_title='frames', yaxis_title='m/(s^3)')
    #figvel.show()
    #figacc.show()
    #figjerk.show()

    # count the number of points in each pyramidal region
    pyramid_forceList = [[] for _ in range(40)]
    IdxByPyramid = [[] for _ in range(40)]
    pyramid_number = -1

    #JointAngle_listoflists = jointAngleCalculations(refined_df, right)
    #points_in_pyramid = [(pyramid_number := pyramid_number + 1, sum(is_inside_pyramid(point, pyramid, pyramid_number, pyramid_forceList, point_number, forces) for point in wristMotionOnly)) for pyramid in unique_pyramids]
    #FIGURE OUT HOW TO MAKE XB version here?
    points_in_pyramid = [
        (
            pyramid_number := pyramid_number + 1,
            sum(
                (is_inside_pyramid(point, pyramid, pyramid_number, pyramid_forceList, point_number, forces, IdxByPyramid, radius)) 
                for point_number, point in enumerate(wristMotionOnly, start=1)
            )
        )
        for pyramid in unique_pyramids
    ]
    #parse and categorize the listoflists for joint angles, velocity, and acceleration
    SynergyTotalSummary, TotalDataAngle_summary, ElbowAngles, ShoulderAngles, WristAngles = jointAngleCalculations(df, right)
    #rint(f'Synergy Correlation Summary: {subject} {SynergyTotalSummary}')

    #print("JOINT ANGLE Jialin method")
    jointangles = calc_jointangles(df)
    jointAngle_list.append(jointangles)

    JointAngle_listoflists = []
    Synergy_listoflists = []
    Vel_byzone = []
    Accel_byzone = []
    points_in_pyramid40 = [[] for _ in range(40)]
    for i in range(40):
        idx = IdxByPyramid[i]
        points_in_pyramid40[i] = len(idx)
        valid_idx = [i for i in idx if i in df.index] #some are dropped as outliers previously - in python, it leaves the index value empty
        invalid_idx = [i for i in idx if i not in df.index]
        idx = valid_idx
        refined_df = df.loc[idx, :]
        vel_curr = veloc[idx[:-1]]
        accel_curr = accel[idx[:-2]]
        vel_summary = distCalculations(vel_curr)
        accel_summary = distCalculations(accel_curr)
        
        if len(idx) > 4:
            print(i)
            JointCorrelation_summary, JointAngle_summary, na, na, na = jointAngleCalculations(refined_df, right)
        else:
            JointAngle_summary = []
            JointCorrelation_summary = []
        JointAngle_listoflists.append(JointAngle_summary)
        Synergy_listoflists.append(JointCorrelation_summary)
        Vel_byzone.append(vel_summary)
        Accel_byzone.append(accel_summary)
        
    points_refined_pyramid = [selectedPyramidPoints[1] for selectedPyramidPoints in points_in_pyramid]
    points_in_pyramid = points_refined_pyramid
    # print the counts
    forcedistributions_ListofLists = []
    #for i, count in enumerate(points_in_pyramid40, 1):
        #print(f"Pyramid {i}: {count} points inside")
    
    for eachPyramidForces in pyramid_forceList:
        if len(eachPyramidForces) > 4:
            eachPyramidForces = np.round(eachPyramidForces, decimals=5)
            median = np.median(eachPyramidForces)
            mode = stats.mode(eachPyramidForces)[0][0]  # mode returns a ModeResult object
            data_range = np.ptp(eachPyramidForces)  # ptp = peak to peak (max - min)
            variance = np.var(eachPyramidForces, ddof=1)  # ddof=1 for sample variance
            std_dev = np.std(eachPyramidForces, ddof=1)  # ddof=1 for sample standard deviation
            iqr = stats.iqr(eachPyramidForces)
            if len(eachPyramidForces) > 4:
                force_skewness = skew(eachPyramidForces)
                #Leptokurtosis
                force_excess_kurtosis = kurtosis(eachPyramidForces, fisher=False)
                #Normal
                ks_test = stats.kstest(eachPyramidForces, 'norm', args=(np.mean(eachPyramidForces), np.std(eachPyramidForces)))
                shapiro_test = stats.shapiro(eachPyramidForces)
                force_ks = ks_test.statistic
                force_shap = shapiro_test.statistic
                expon_stat, p_exp_ks = stats.kstest(eachPyramidForces, 'expon')
                lognormal_stat, p_lognormal_sw = stats.shapiro(np.log(eachPyramidForces))
            else:
                force_skewness = np.nan
                force_excess_kurtosis = np.nan
                force_ks = np.nan
                force_shap = np.nan
                expon_stat = np.nan
                lognormal_stat = np.nan
            
            forcedistributions_ListofLists.append([median, mode, data_range, std_dev, iqr, force_skewness, force_excess_kurtosis, len(eachPyramidForces)])
        else:
            force_skewness = np.nan
            force_excess_kurtosis = np.nan
            force_ks = np.nan
            force_shap = np.nan
            expon_stat = np.nan
            lognormal_stat = np.nan
            forcedistributions_ListofLists.append([])

    #MOVEMENT DIVERSITY ANALYSIS
    #Find the angular velocities and accelerations
    Vel_ShoulderAngle = np.diff(ShoulderAngles) * fs
    Vel_ElbowAngle = np.diff(ElbowAngles) * fs
    Vel_WristAngle = np.diff(WristAngles) * fs
    Acc_ShoulderAngle = np.diff(Vel_ShoulderAngle) * fs
    Acc_ElbowAngle = np.diff(Vel_ElbowAngle) * fs
    Acc_WristAngle = np.diff(Vel_WristAngle) * fs
    zoned_startpts = [[] for i in range(40)]
    zoned_endzones = [[] for i in range(40)]
    Mvt_linearvelocities = []
    Mvt_ShoulAngularVel = []
    Mvt_ElbowAngularVel = []
    Mvt_WristAngularVel = []
    Mvt_linearaccelerations = []
    Mvt_ShoulAngularAcc = []
    Mvt_ElbowAngularAcc = []
    Mvt_WristAngularAcc = []
    Mvt_TADshoulder = []
    Mvt_NADshoulder = []
    Mvt_TADelbow = []
    Mvt_NADelbow = []
    all_startpts = adj_trough_indices #split by jerk
    for s, startpt in enumerate(all_startpts, start=1):
        if s < len(all_startpts):
            endpt = all_startpts[s] - 1 #find the start and end point for each movement - ignore last one
            for i, sublist in enumerate(IdxByPyramid):
                if startpt in sublist:
                    zoned_startpts[i].append(startpt)
                    for j, endzoneList in enumerate(IdxByPyramid, start=1):
                        if endpt in endzoneList:
                            zoned_endzones[i].append(j)
                            break
                    break
            #calculate the net angular displacement, total angular displacement + ratio of the two per movement
            # Calculate total angular displacement
            currMvt_ShAng = ShoulderAngles[startpt:endpt]
            tad_curr_Shoulder = np.sum(np.abs(np.diff(currMvt_ShAng)))
            nad_curr_Shoulder = currMvt_ShAng[-1] - currMvt_ShAng[0]
            Mvt_TADshoulder.append(tad_curr_Shoulder)
            Mvt_NADshoulder.append(nad_curr_Shoulder)
            currMvt_ElbAng = ElbowAngles[startpt:endpt]
            tad_curr_Elbow = np.sum(np.abs(np.diff(currMvt_ElbAng)))
            nad_curr_Elbow = currMvt_ElbAng[-1] - currMvt_ElbAng[0]
            Mvt_TADelbow.append(tad_curr_Elbow)
            Mvt_NADelbow.append(nad_curr_Elbow)
            #linear accelerations - not using this rn
            currMvt_vel = veloc[startpt:endpt]
            currMvt_velAvg = np.mean(currMvt_vel)
            Mvt_linearvelocities.append(currMvt_velAvg)
            currMvt_acc = accel[startpt:endpt]
            currMvt_accAvg = np.mean(currMvt_acc)
            Mvt_linearaccelerations.append(currMvt_accAvg)
            #angular velocities
            currMvt_ShAngularVel = Vel_ShoulderAngle[startpt:endpt]
            #median shoulder angular vel and acc
            if len(currMvt_ShAngularVel) > 5:
                currMvt_ShAngVelMed = np.median(currMvt_ShAngularVel)
                Mvt_ShoulAngularVel.append(currMvt_ShAngVelMed)
                currMvt_ShAngularAcc = Acc_ShoulderAngle[startpt:endpt]
                currMvt_ShAngAccMed = np.median(currMvt_ShAngularAcc)
                Mvt_ShoulAngularAcc.append(currMvt_ShAngAccMed)
            #elbow angular vel and acc
            currMvt_ElAngularVel = Vel_ElbowAngle[startpt:endpt]
            if len(currMvt_ElAngularVel)> 5:
                currMvt_ElAngVelMed = np.median(currMvt_ElAngularVel)
                Mvt_ElbowAngularVel.append(currMvt_ElAngVelMed)
                currMvt_ElAngularAcc = Acc_ElbowAngle[startpt:endpt]
                currMvt_ElAngAccMed = np.median(currMvt_ElAngularAcc)
                Mvt_ElbowAngularAcc.append(currMvt_ElAngAccMed)
            #wrist angular vel and acc
            currMvt_WriAngularVel = Vel_WristAngle[startpt:endpt]
            if len(currMvt_WriAngularVel) > 5:
                currMvt_WriAngularVelMed = np.median(currMvt_WriAngularVel)
                Mvt_WristAngularVel.append(currMvt_WriAngularVelMed)
                currMvt_WriAngularAcc = Acc_WristAngle[startpt:endpt]
                currMvt_WriAngAccMed = np.median(currMvt_WriAngularAcc)
                Mvt_WristAngularAcc.append(currMvt_WriAngAccMed)
                
    #print('CHECK movement selection')
    #print(zoned_startpts)
    #print(zoned_endzones)

    #plot scatter for overall shoulder and elbow angular velocities and accelerations
    #fig = go.Figure(data=go.Scatter(x=np.abs(Vel_ShoulderAngle), y=np.abs(Vel_ElbowAngle), mode='markers'))
    #fig.update_layout(title='Shoulder and Elbow Angular Velocities', xaxis_title='Shoulder deg/s', yaxis_title='Elbow deg/s')
    #fig.show()

    #plot scatter for overall shoulder and elbow angular velocities and accelerations
    #fig = go.Figure(data=go.Scatter(x=np.abs(Acc_ShoulderAngle), y=np.abs(Acc_ElbowAngle), mode='markers'))
    #fig.update_layout(title='Shoulder and Elbow Angular Accelerations', xaxis_title='Shoulder deg/(s^2)', yaxis_title='Elbow deg/(s^2)')
    #fig.show()

    #plot scatter of total angular displacement and net angular displacement by movement -underlying synergies?
    #x_indices = list(range(len(Mvt_TADshoulder)))
    #tad_shoulder = go.Scatter(x=x_indices, y=Mvt_TADshoulder, mode='markers', name='TAD Shoulder', marker=dict(color='blue'))
    #nad_shoulder = go.Scatter(x=x_indices, y=Mvt_NADshoulder, mode='markers', name='NAD Shoulder', marker=dict(color='green'))
    #fig = go.Figure(data=[tad_shoulder, nad_shoulder])
    ##fig.update_layout(title='tad and nad shoulder angles per movement', xaxis_title='Movement #', yaxis_title='Angular Displacement (deg)')
    #figfilename = f'{subject}_shoulder_tadnad.png'
    #fig.write_image(figfilename)

    #x_indices = list(range(len(Mvt_TADelbow)))
    #tad_elbow = go.Scatter(x=x_indices, y=Mvt_TADelbow, mode='markers', name='TAD Elbow', marker=dict(color='blue'))
    #nad_elbow = go.Scatter(x=x_indices, y=Mvt_NADelbow, mode='markers', name='NAD Elbow', marker=dict(color='green'))
    #fig = go.Figure(data=[tad_elbow, nad_elbow])
    #fig.update_layout(title='tad and nad elbow angles per movement', xaxis_title='Movement #', yaxis_title='Angular Displacement (deg)')
    #figfilename = f'{subject}_elbow_tadnad.png'
    #fig.write_image(figfilename)


    #Convhull positon data
    WristPositionhull = ConvexHull(wristMotionOnly)
    #plt.plot(wristMotionOnly[:, 0], wristMotionOnly[:, 1], 'o')
    #for simplex in WristPositionhull.simplices:
    #    plt.plot(wristMotionOnly[simplex, 0], wristMotionOnly[simplex, 1], 'k-')
    #plt.show()
    print(f'Convex Hull Wrist Position 3D volume {WristPositionhull.area}')

    #adjacency matrix - to show node connections that represent the zones
    
    ## count the number of points in each pyramidal region
    #points_in_pyramid = [sum(is_inside_pyramid(point, pyramid) for point in wristMotionOnly) for pyramid in unique_pyramids]
    
    # print the counts
   # for i, count in enumerate(points_in_pyramid, 1):
    #    print(f"Pyramid {i}: {count} points inside")
    
    # call the function to plot the icosahedron
    #plot_icosahedron(unique_pyramids, points_in_pyramid, wristMotionOnly, subject)

    # Rectilinear modeling - center +/- radius in x,y,z direction - divide each dimension by ndim
    n_bins = 3
    xb, bin_limits, bin_centers = fixing_bin_center(wristMotionOnly, n_bins, center, right)
    #print(bin_centers)
    #print(bin_limits)

    #convert x,y,z index to rectilinear S,R,C indexing
    regFinder = np.full(xb.shape, np.nan)  # NaN array like MATLAB NaN*X
    regFinder[:, 2] = 2 - xb[:, 0] #x-column convert to column-column (2-x bc inverted indexing)
    regFinder[:, 1] = 2 - xb[:, 1]  #y-column convert to rows-column (2-y bc inverted indexing)
    regFinder[:, 0] = 2 - xb[:, 2] #z-column convert to slice-column (2-z bc inverted indexing)
    
    Nxb, ndim = regFinder.shape
    rect_forceList = [[[[] for _ in range(n_bins)] for _ in range(n_bins)] for _ in range(n_bins)]
    #Organize list of lists for forces based on regFinder
    for l in (range(Nxb - 2)):
        currForce_designation = regFinder[l, :]
        currForce_designation = [int(i) for i in currForce_designation]
        rect_forceList[currForce_designation[0]][currForce_designation[1]][currForce_designation[2]].append(forces[l])
    
    #Calculate average forces per bin - rectilinear
    rect_force_avgs =  [[[[] for _ in range(n_bins)] for _ in range(n_bins)] for _ in range(n_bins)]
    # Iterate through each 2D sub-list in the 3x3x3 list
    RectForceDistributions_ListofLists =  [[[[] for _ in range(n_bins)] for _ in range(n_bins)] for _ in range(n_bins)]
    for i in range(len(rect_forceList)):
        for j in range(len(rect_forceList[i])):
            for k in range(len(rect_forceList[i][j])):
                
                if len(rect_forceList[i][j][k]) > 0:
                # Calculate the average of the 1D sub-list
                    avg = np.mean(rect_forceList[i][j][k])
                    avg = np.round(avg, decimals=5)
                # Replace the 1D sub-list with the average
                    rect_force_avgs[i][j][k] = avg
                    if len(rect_forceList[i][j][k]) > 4:
                        force_skewness = skew(rect_forceList[i][j][k])
                        #Leptokurtosis
                        force_excess_kurtosis = kurtosis(rect_forceList[i][j][k], fisher=False)
                    #Normal
                        ks_test = stats.kstest(rect_forceList[i][j][k], 'norm', args=(np.mean(rect_forceList[i][j][k]), np.std(rect_forceList[i][j][k])))
                        shapiro_test = stats.shapiro(rect_forceList[i][j][k])
                        force_ks = ks_test.statistic
                        force_shap = shapiro_test.statistic
                        expon_stat, p_exp_ks = stats.kstest(rect_forceList[i][j][k], 'expon')
                        lognormal_stat, p_lognormal_sw = stats.shapiro(np.log(rect_forceList[i][j][k]))
                    else:
                        force_skewness = np.nan
                        force_excess_kurtosis = np.nan
                        force_ks = np.nan
                        force_shap = np.nan
                        expon_stat = np.nan
                        lognormal_stat = np.nan
                    RectForceDistributions_ListofLists[i][j][k] = [avg, force_skewness, force_excess_kurtosis, force_ks, force_shap, expon_stat, lognormal_stat, len(rect_forceList[i][j][k])]
          
                else:
                    rect_force_avgs[i][j][k] = 0
                    force_skewness = np.nan
                    force_excess_kurtosis = np.nan
                    force_ks = np.nan
                    force_shap = np.nan
                    expon_stat = np.nan
                    lognormal_stat = np.nan
                    RectForceDistributions_ListofLists[i][j][k] = [0, force_skewness, force_excess_kurtosis, force_ks, force_shap, expon_stat, lognormal_stat, 0]

    I_list = []
    results_rectilinear = np.zeros((n_bins, n_bins, n_bins), dtype = int)
    #make a results array that displays number in each bin
    for j1 in range(0, n_bins):
        for j2 in range(0, n_bins):
            for j3 in range(0, n_bins):
                comp = np.ones((Nxb, 3)) * np.array([j1, j2, j3])
                match = np.all(comp == regFinder, axis=1)
                Sums = np.sum(match)
                results_rectilinear[j1, j2, j3] = Sums
                if Sums > 0:
                    I = np.where(match)[0]
                    I_list.extend(I)
    
    I = np.array(I_list)
    #print(results_rectilinear)

    # flatten all layers of the data array
    flattened_data = results_rectilinear.flatten()
    #print(flattened_data)
    
    # generate colors based on the flattened data (using global normalization)
    colors = generate_colors(flattened_data)
    #print(colors)
    
    # reshape the colors array to match the shape of the data array
    colors = np.array(colors).reshape((3, 3, 3, 4))
    #print(colors)
    
    # plotting
    fig3, axs = plt.subplots(1, 3, figsize=(6, 2))
    
    #for i in range(3):
    #    axs[i].imshow(colors[i, :, :, :], aspect='auto')
    #    axs[i].set_xticks([])
    #    axs[i].set_yticks([])
    #    axs[i].set_title(f'Layer {i}')
    
    #plt.tight_layout()
    #plt.show()
    
    #Stats for the participant
    total_samples = sum(points_in_pyramid40)
    percentTime = [round(point_in_pyramid / total_samples * 100, 2) for point_in_pyramid in points_in_pyramid40]
    #print(percentTime)
    #total_mvts = sum(startpt_counts)
    #percentStartMvt = [round(startpt_count / total_mvts * 100, 2) for startpt_count in startpt_counts]
    
    #Update the Main Hubs for stats
    stats_icosahedron[subject] = percentTime

    #stats_vel_icos[subject] = Vel_byzone
    #stats_acc_icos[subject] = Accel_byzone

    #stats_forceavg_icosahedron[subject] = forcedistributions_ListofLists
    #stats_jointAngavg_icosahedron[subject] = JointAngle_listoflists
    #stats_synergy_icosahedron[subject] = Synergy_listoflists
    #stats_mvt_icosahedron[subject] = percentStartMvt

    #Stats for rectilinear
    total_samples_rectilinear = np.sum(results_rectilinear)
    percentTime_rectilinear = [np.round(result / total_samples_rectilinear * 100, decimals=2) for result in results_rectilinear]
    #print(percentTime_rectilinear)
    #Update the Main Hubs for stats
    percentTime_rectilinear = np.array(percentTime_rectilinear)
    stats_rectilinear[subject] = percentTime_rectilinear.flatten()
    #print('rectilinear stats')
    #print(stats_rectilinear[subject])
    #Force Rectilinear Stats
    #flat_forcedistributions_list = [item for sublist1 in RectForceDistributions_ListofLists for sublist2 in sublist1 for item in sublist2]
    #stats_forceavg_rectilinear[subject] = flat_forcedistributions_list

    #summaryLists = [WristPositionhull.area, SynergyTotalSummary[0][0]]
    #stats_summary[subject] = summaryLists
    
    return jointAngle_list, stats_summary, stats_icosahedron, stats_rectilinear, stats_vel_icos, stats_acc_icos, stats_forceavg_icosahedron, stats_forceavg_rectilinear, stats_jointAngavg_icosahedron, stats_synergy_icosahedron, stats_mvt_icosahedron


In [ ]:
# %% Plot 3D figures
def generate_circle_points(centroid, normal, radius=0.1, num_points=50):
    # Create a vector perpendicular to the normal
    if normal[0] == 0 and normal[1] == 0:
        perp_vector = np.array([1, 0, 0])
    else:
        perp_vector = np.cross(normal, np.array([0, 0, 1]))
    
    # Normalize the perpendicular vector
    perp_vector = perp_vector / np.linalg.norm(perp_vector)
    
    # Generate points on the circle in the plane perpendicular to the normal
    circle_points = []
    for theta in np.linspace(0, 2 * np.pi, num_points):
        # Calculate position in 3D space
        point = (
            centroid +
            radius * np.cos(theta) * perp_vector +
            radius * np.sin(theta) * np.cross(normal, perp_vector)
        )
        circle_points.append(point)
    return np.array(circle_points)
    
def generate_fixed_positions_on_face(vertices):
    """
    Generates fixed positions for 19 participants on a triangular face:
    - 3 participants on each median (3 medians = 9),
    - 1 at the centroid,
    - 3 additional participants in each of the 3 regions between the medians (9 total).
    """
    # Calculate the centroid
    centroid = np.mean(vertices, axis=0)

    # Get the midpoints of each median
    midpoints = []
    for i in range(3):
        start_vertex = vertices[i]
        end_vertex = centroid  # The centroid is the end point of each median
        midpoints.append(start_vertex + (end_vertex - start_vertex) * (1/4))  # 1/4 of the way to the centroid
        midpoints.append(start_vertex + (end_vertex - start_vertex) * (2/4))  # 2/4 of the way to the centroid
        midpoints.append(start_vertex + (end_vertex - start_vertex) * (3/4))  # 3/4 of the way to the centroid

    # Combine the midpoints and centroid
    positions = [centroid] + midpoints

    # Now add 3 points in each region between the medians
    # Get the start vertex and the centroid
    restpoints= []
    for i in range(3):
        m1 = midpoints[3 * i]  # Midpoint at 1/2
        m2 = midpoints[(3 * ((i + 1) % 3))]  # Midpoint at 1/2 of the next median
        m3 = midpoints[3*i + 1]
        m4 = midpoints[3 * ((i + 1) % 3) + 1]  # Midpoint at 3/4

        # Add points in the regions between the median points
        restpoints.append(m1 + (m2 - m1) * (1/3))  # 1/3 between m1 and m2
        restpoints.append(m1 + (m2 - m1) * (2/3))  # 2/3 between m1 and m2
        restpoints.append(m3 + (m4 - m3) * (1/2))  # Midpoint between m1 and m3

    # Combine all positions
    positions += restpoints

    return np.array(positions)

def generatePointer(vertices):
    scale_factor = 0.1
    #calculate the centroid
    centroid = np.mean(vertices, axis=0)

    trianglePointer = (vertices - vertices[0]) * scale_factor + vertices[0]

    return np.array(trianglePointer)

def plot3dIcosahedron(arr_main, title, colorbar_title, array_unique_pyramids, nt_ci_data, small, arr_comp=None):
    # Icosahedron neurotypical average
    colors = generate_colors(arr_main, arr_comp)
    # convert colors to Plotly-compatible RGBA strings
    rgba_colors = ['rgba({}, {}, {}, {})'.format(int(color[0]*255), int(color[1]*255), int(color[2]*255), color[3]) for color in colors]
    # print the length of rgba_colors
    print("Shape of rgba_colors:", len(rgba_colors))
    df_icosahedronSummary = pd.read_excel('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/StatisticsCollection/ZoneDistribution_Icosahedron_r2.xlsx')
    # Check the variability per Icosahedron zone of the neurotypicals
    s_columns = [col for col in df_icosahedronSummary.columns if col.startswith('s')]
    p_columns = [col for col in df_icosahedronSummary.columns if col.startswith('p')]

    # load the OBJ file and parse its contents - PERSON FIGURINE
    vertices = []
    faces = []
    with open('/Users/adithsrivatsa/Downloads/Man_Body_Valentino.obj', 'r') as file:
        for line in file:
            if line.startswith('v '):
                # parse vertex coordinates (x, y, z)
                vertex = list(map(float, line.strip().split()[1:]))
                temp = vertex[0]
                vertex[0] = (-1/300*0.65 * vertex[1]) + 0.15
                vertex[1] = (1/660*0.65 * vertex[2]) - 1.35
                vertex[2] = (-1/600*0.65 * temp) + 0.25
                vertices.append(vertex)
            elif line.startswith('f '):
                # parse face indices (assuming triangular faces)
                face_elements = line.strip().split()[1:]
                face_indices = []
                for element in face_elements:
                    # parse the vertex index (ignore texture and normal indices)
                    vertex_index = int(element.split('/')[0]) - 1
                    face_indices.append(vertex_index)
                faces.append(face_indices)

    #make the centroid faces
    # Calculate face centroids
    face_centroids = []
    for face in range(len(array_unique_pyramids)):
        x_coords = array_unique_pyramids[face, 1:, 0]
        y_coords = array_unique_pyramids[face, 1:, 1]
        z_coords = array_unique_pyramids[face, 1:, 2]
        centroid = (
            np.mean(x_coords),
            np.mean(y_coords),
            np.mean(z_coords)
        )
        face_centroids.append(centroid)

    #calculate the normals for the corresponding faces
    face_normals = []
    for face in range(len(array_unique_pyramids)):
        # Get coordinates of three vertices of the face
        v0 = array_unique_pyramids[face, 1]
        v1 = array_unique_pyramids[face, 2]
        v2 = array_unique_pyramids[face, 3]
        # Calculate two edge vectors of the face
        edge1 = v1 - v0
        edge2 = v2 - v0
        # Compute the cross product of the edge vectors to get the normal vector
        normal = np.cross(edge1, edge2)
        # Normalize the normal vector to get the unit direction vector
        normal = normal / np.linalg.norm(normal)
        # Append the normal vector to the list
        face_normals.append(normal)

    
    #vertices and faces defined before 3D plots begin
    # convert vertices and faces to numpy arrays
    vertices = np.array(vertices)
    faces = np.array(faces)
    #define tick bounds
    if arr_comp is None: 
        max_abs = max(abs(num) for num in arr_main)
    else:
        max_abs = max(max(abs(num) for num in arr_main), max(abs(num) for num in arr_comp))


    ticklow = -math.ceil(max_abs)
    tickmid = 0
    tickhigh = math.ceil(max_abs)

    diff_sample= arr_main - arr_main + 0.2 # make a test array of len(arr_main) but with the same value throughout
    stroke_df = df_icosahedronSummary[p_columns]
    if small == 1:
        stroke_df = stroke_df.head(20)
    else:
        stroke_df = stroke_df.tail(20)
    print(f'Stroke Data Shape {np.shape(stroke_df)}')
    print(f'Stroke Data {stroke_df}')
    
    fig = go.Figure()
        # Create the figurine dude
    #fig.add_trace(go.Mesh3d(
    #    x=vertices[:, 0],  # X coordinates of vertices
    #    y=vertices[:, 1],  # Y coordinates of vertices
    #    z=vertices[:, 2],  # Z coordinates of vertices
    #    i=faces[:, 0],  # Indices of first vertex of each face
    #    j=faces[:, 1],  # Indices of second vertex of each face
    #    k=faces[:, 2],  # Indices of third vertex of each face
    #    color='gray',  # Color of the figurine
    #    opacity=1,
    #    flatshading=True,
    #))

    #fig.add_trace(go.Scatter3d(
    #    x=vertices[:, 0],  # X coordinates of vertices
    #    y=vertices[:, 1],  # Y coordinates of vertices
    #    z=vertices[:, 2],  # Z coordinates of vertices
    #    mode='markers',
    #    marker=dict(
    #        size=4,
    #        color='black',                # set color to an array/list of desired values
    #        opacity=1
    #    )
    #))
    
    #officially add in the mesh for each face of the pyramid
    num_subjects = 19  # Assuming 19 participants
    for face in range(len(array_unique_pyramids)):
        #print(f"Face {face+1}")
        x_coords = array_unique_pyramids[face, 1:, 0] * 0.99
        y_coords = array_unique_pyramids[face, 1:, 1] * 0.99
        z_coords = array_unique_pyramids[face, 1:, 2] * 0.99
        print(f' Face coordinates {face + 1}: {array_unique_pyramids[face]}')
        i = [0, 1, 2]  # indices for vertices of the triangle
        j = [1, 2, 0]
        k = [2, 0, 1]
        
        vertices = array_unique_pyramids[face, 1:]
         # Generate fixed positions on the face
        fixed_positions = generate_fixed_positions_on_face(vertices)
        zone_data = stroke_df.iloc[face]
        #print(f'Zone Data: {zone_data}')
        #add a black/gold triangle mesh 1/10 size of the face that points to the top triangle
        trianglePointer = generatePointer(vertices)
        #print(f'Face {face+1}: trianglePointer - {trianglePointer}')
        fig.add_trace(go.Mesh3d(
            x=trianglePointer[:, 0],
            y=trianglePointer[:, 1],
            z=trianglePointer[:, 2],
            color='goldenrod',
            opacity=1))
        # Add a text label at a specific position
        x_c, y_c, z_c = fixed_positions[0]
        fig.add_trace(
            go.Scatter3d(
                x=[x_c* 1.02],  # X coordinate of the text
                y=[y_c * 1.02],  # Y coordinate of the text
                z=[z_c * 1.02],  # Z coordinate of the text
                text=face+1,  # The text value
                mode='text',  # Display only text
                textfont=dict(
                    size=12,
                    color='black'
                )
            )
        )
            
        #print(f'Zone Data Size {np.shape(zone_data)}')
        # For each subject, plot circles on the faces
        for subject_idx, subject_value in enumerate(zone_data):
        #indent this when u add the subject_idx
            #subject_value = subject_values[face]  # Get the value for the current face - change to this when u have different values for each
            # Generate circle points around the centroid
            # Get the fixed position for this subject
            if subject_idx < 19: # FIX THIS
                fixed_position = fixed_positions[subject_idx]
            x_c, y_c, z_c = fixed_position  # Get the centroid of the current face
            normal = face_normals[face]  # Get the normal for the current face
            #bring back the nt values to calculate radius and define the circle fill
            ci_df = pd.DataFrame(nt_ci_data)
            if small == 0:
                ci_lower = (ci_df['CI Lower'][face+20] - 1e-6) * 100
                #print(f'ci Lower: {ci_lower}')
                ci_upper = (ci_df['CI Upper'][face+20] - 1e-6) * 100
                #print(f'ci upper: {ci_upper}')
                meanVal = (ci_df['Mean'][face+20]- 1e-6) * 100
            else: 
                ci_lower = (ci_df['CI Lower'][face] - 1e-6) * 100
                #print(f'ci Lower: {ci_lower}')
                ci_upper = (ci_df['CI Upper'][face] - 1e-6) * 100
                #print(f'ci upper: {ci_upper}')
                meanVal = (ci_df['Mean'][face] - 1e-6) * 100
            valueDiff = np.abs(subject_value - meanVal)
            if small == 1:
                radius = (0.002 + np.sqrt(valueDiff) * 0.02) / 2
            else:
                radius = 0.002 + np.sqrt(valueDiff) * 0.02
            circle_points = generate_circle_points(np.array([x_c, y_c, z_c]), normal, radius=radius)
        
            # Separate circle points into x, y, z for plotting
            x_circle = circle_points[:, 0]
            y_circle = circle_points[:, 1]
            z_circle = circle_points[:, 2]
        

            if subject_value > ci_upper:
                #print('red')
                fill_color = 'red'
                fig.add_trace(go.Mesh3d(
                    x=x_circle, y=y_circle, z=z_circle,
                    color=fill_color,
                    opacity=1,
                    name=f'Filled red circle for subject {subject_idx + 1}',
                    delaunayaxis='z' 
                ))
                border_color = 'darkred'
                fig.add_trace(go.Scatter3d(
                    x=x_circle, y=y_circle, z=z_circle,
                    mode='lines',
                    line=dict(color=border_color, width=4),  # Adjust line width if needed
                    name=f'Border for subject {subject_idx + 1}'
                ))
                #fig.add_trace(go.Scatter3d(
                #    x=x_circle, y=y_circle, z=z_circle,
                #    mode='lines',
                #    line=dict(color='red', width=6),
                #    name=f'Circle around centroid {face + 1}'
                #))
            elif subject_value < ci_lower:
                #print('blue')
                fill_color = 'blue'
                fig.add_trace(go.Mesh3d(
                    x=x_circle, y=y_circle, z=z_circle,
                    color=fill_color,
                    opacity=1,
                    name=f'Filled blue circle for subject {subject_idx + 1}',
                    delaunayaxis='z' 
                ))
                border_color = 'darkblue'
                fig.add_trace(go.Scatter3d(
                    x=x_circle, y=y_circle, z=z_circle,
                    mode='lines',
                    line=dict(color=border_color, width=4),  # Adjust line width if needed
                    name=f'Border for subject {subject_idx + 1}'
                ))
            else:
            #Add the circle trace to the plot
                #print('open')
                fig.add_trace(go.Scatter3d(
                    x=x_circle, y=y_circle, z=z_circle,
                    mode='lines',
                    line=dict(color='black', width=2),
                    name=f'Circle for subject {subject_idx + 1}'
                ))
    
        if arr_comp is None: 
              fig.add_trace(go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i,
                j=j,
                k=k,
                intensity=[1, 1, 1],  # Set intensity to a single value
                colorscale=[[0, 'white'], [1, 'white']],  # Make all faces white
                opacity=1,
                intensitymode='cell',
                showscale=False  # Remove the colorbar
            ))
        else:
           fig.add_trace(go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i,
                j=j,
                k=k,
                intensity=[1, 1, 1],  # Set intensity to a single value
                colorscale=[[0, 'white'], [1, 'white']],  # Make all faces white
                opacity=1,
                intensitymode='cell',
                showscale=False  # Remove the colorbar
            ))
    
        for edge in range(3):
            fig.add_trace(go.Scatter3d(
                x=[x_coords[edge], x_coords[(edge + 1) % 3]],
                y=[y_coords[edge], y_coords[(edge + 1) % 3]],
                z=[z_coords[edge], z_coords[(edge + 1) % 3]],
                mode='lines',
                line=dict(color='black', width=2)
            ))

        # Update layout total
    fig.update_layout(
        title=title,
        scene=dict(
            aspectmode='data',
            xaxis=dict(title='X Axis'),
            yaxis=dict(title='Y Axis'),
            zaxis=dict(title='Z Axis')
        ),
        width=800,
        height=800,
        showlegend=False,
        margin=dict(l=50, r=50, b=50, t=50),
    )

        
    #clear out the background color
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        plot_bgcolor='white'
    )
    # Show the figure
    fig.show()

def generate_circle_positions(x_rect, y_rect, z_rect, z_offset=-0.01):
    # Calculate the bounds of the square
    x_min, x_max = min(x_rect), max(x_rect)
    y_min, y_max = min(y_rect), max(y_rect)
    z_fixed = np.mean(z_rect) + z_offset  # Slightly in front of the plane

    # Define a 4x5 grid (20 positions, use only 19)
    x_grid = np.linspace(x_min+0.05, x_max-0.05, 5)
    y_grid = np.linspace(y_min+0.05, y_max-0.05, 4)
    x_positions, y_positions = np.meshgrid(x_grid, y_grid)
    
    # Flatten the grid and select the first 19 positions
    x_flat = x_positions.flatten()[:19]
    y_flat = y_positions.flatten()[:19]
    z_flat = [z_fixed] * 19  # Same z-offset for all circles

    # Combine into a list of tuples
    positions = list(zip(x_flat, y_flat, z_flat))
    #print(f'circle positions: {positions}')
    return positions

def plot3dRectilinear(stroke_diff_df, Rect_nt_ci_data, title, arr_comp=None):

    # load the OBJ file and parse its contents - PERSON FIGURINE
    vertices = []
    faces = []
    with open('/Users/adithsrivatsa/Downloads/Man_Body_Valentino.obj', 'r') as file:
        for line in file:
            if line.startswith('v '):
                # parse vertex coordinates (x, y, z)
                vertex = list(map(float, line.strip().split()[1:]))
                temp = vertex[0]
                vertex[0] = (-1/300*0.65 * vertex[1]) + 0.15
                vertex[1] = (1/660*0.65 * vertex[2]) - 1.35
                vertex[2] = (-1/600*0.65 * temp) + 0.25
                vertices.append(vertex)
            elif line.startswith('f '):
                # parse face indices (assuming triangular faces)
                face_elements = line.strip().split()[1:]
                face_indices = []
                for element in face_elements:
                    # parse the vertex index (ignore texture and normal indices)
                    vertex_index = int(element.split('/')[0]) - 1
                    face_indices.append(vertex_index)
                faces.append(face_indices)
    vertices = np.array(vertices)
    faces = np.array(faces)
    
    fig = go.Figure()
    # Define the coordinates for each bin
    #test s08 and get max bin_limits
    #print("Shape of bin_limits", np.shape(bin_limits))
    count = 0
    for zi in range(2, -1, -1):
        for yi in range(2, -1, -1):
            for xi in range(2, -1, -1):
                # find the vertices for each rectangle
                x_rect = []
                y_rect = []
                z_rect = []
                i_rect = []
                j_rect = []
                k_rect = []
                
                x_rect.extend([bin_limits[xi, 0], bin_limits[xi + 1, 0], bin_limits[xi + 1, 0], bin_limits[xi, 0]])
                y_rect.extend([bin_limits[yi, 1], bin_limits[yi, 1], bin_limits[yi + 1, 1], bin_limits[yi + 1, 1]])
                z_rect.extend([bin_limits[zi, 2], bin_limits[zi, 2], bin_limits[zi, 2], bin_limits[zi, 2]])
    
                #plotly works with triangular faces so find these
                vertex_counter = 0
                i_rect.extend([vertex_counter, vertex_counter, vertex_counter + 1])
                j_rect.extend([vertex_counter + 1, vertex_counter + 2, vertex_counter + 2])
                k_rect.extend([vertex_counter + 2, vertex_counter + 3, vertex_counter + 3])


                #Plot the mesh in place?
                fig.add_trace(go.Mesh3d(
                    x=x_rect,
                    y=y_rect,
                    z=z_rect,
                    i=i_rect,
                    j=j_rect,
                    k=k_rect,
                    intensity=[1, 5, 10],
                    colorscale=[[0, 'white'], [1, 'white']],
                    intensitymode ='cell',
                ))

                #plot circles for each person
                circle_positions = generate_circle_positions(x_rect, y_rect, z_rect)
                for col in range(stroke_diff_df.shape[1]):
                    position = circle_positions[col]
                    x_c, y_c, z_c = position
                    #radius = stroke_diff_df.iloc[2, col] #same radius for everything for now
                    curr_val = stroke_diff_df.iloc[count, col]
                    radius = 0.002 + np.sqrt(np.abs(curr_val * 100)) * 0.01
                    theta = np.linspace(0, 2 * np.pi, 100)
                    x_circle = x_c + radius * np.cos(theta)
                    y_circle = y_c + radius * np.sin(theta)
                    z_circle = [z_c] * len(theta)

                    ci_upper = Rect_nt_ci_data[count]['CI Upper'] - 1e-6 - Rect_nt_ci_data[count]['Mean']
                    ci_lower = Rect_nt_ci_data[count]['CI Lower'] - 1e-6 - Rect_nt_ci_data[count]['Mean']
                    #print(f'{count} zone: ci upper - {ci_upper}, ci lower - {ci_lower}')

                    if curr_val > ci_upper:
                        #print('red')
                        fill_color = 'red'
                        fig.add_trace(go.Mesh3d(
                            x=x_circle, y=y_circle, z=z_circle,
                            color=fill_color,
                            opacity=1,
                        ))
                        border_color = 'darkred'
                        fig.add_trace(go.Scatter3d(
                            x=x_circle, y=y_circle, z=z_circle,
                            mode='lines',
                            line=dict(color=border_color, width=2),  # Adjust line width if needed
                        ))
                        #fig.add_trace(go.Scatter3d(
                        #    x=x_circle, y=y_circle, z=z_circle,
                        #    mode='lines',
                        #    line=dict(color='red', width=6),
                        #    name=f'Circle around centroid {face + 1}'
                        #))
                    elif curr_val < ci_lower:
                        #print('blue')
                        fill_color = 'blue'
                        fig.add_trace(go.Mesh3d(
                            x=x_circle, y=y_circle, z=z_circle,
                            color=fill_color,
                            opacity=1,
                        ))
                        border_color = 'darkblue'
                        fig.add_trace(go.Scatter3d(
                            x=x_circle, y=y_circle, z=z_circle,
                            mode='lines',
                            line=dict(color=border_color, width=2),  # Adjust line width if needed
                        ))
                    else:
                        #print('open')
                        fig.add_trace(go.Scatter3d(
                            x=x_circle, y=y_circle, z=z_circle,
                            mode='lines',
                            line=dict(color='black', width=2),
                        ))
    
                edges = []
                for i, j, k in zip(i_rect, j_rect, k_rect):
                    #edges.append((x_rect[i], y_rect[i], z_rect[i]))
                    edges.append((x_rect[j], y_rect[j], z_rect[j]))
                    #edges.append((x_rect[j], y_rect[j], z_rect[j]))
                    edges.append((x_rect[k], y_rect[k], z_rect[k]))
                    edges.append((x_rect[k], y_rect[k], z_rect[k]))
                    #edges.append((x_rect[i], y_rect[i], z_rect[i]))
                
                # Plot edges
                fig.add_trace(go.Scatter3d(
                    x=[edge[0] for edge in edges],
                    y=[edge[1] for edge in edges],
                    z=[edge[2] for edge in edges],
                    mode='lines',
                    line=dict(color='black', width=2),
                ))
                count +=1
                #print(count)
    
    
    #Define figurine in this plot
    # Create a Mesh3d trace using the parsed data
    #fig.add_trace(go.Mesh3d(
    #    x=vertices[:, 0],  # X coordinates of vertices
    #    y=vertices[:, 1],  # Y coordinates of vertices
    #    z=vertices[:, 2],  # Z coordinates of vertices
    #    i=faces[:, 0],  # Indices of first vertex of each face
    #    j=faces[:, 1],  # Indices of second vertex of each face
    #    k=faces[:, 2],  # Indices of third vertex of each face
    #    color='grey',  # Color of the figurine
    #    opacity=1,
    #    flatshading=True,
    #))
    
    # Hide axes
    fig.update_layout(
        title=title,
        scene=dict(
            aspectmode='data',
            xaxis=dict(title='X Axis'),
            yaxis=dict(title='Y Axis'),
            zaxis=dict(title='Z Axis')
        ),
        width=800,
        height=800,
        showlegend=False,
        margin=dict(l=50, r=50, b=50, t=50),
    )
    fig.update_layout(
        paper_bgcolor='white',  # Background of the figure area
        scene=dict(
            xaxis=dict(showgrid=False, showbackground=True, backgroundcolor='white'),
            yaxis=dict(showgrid=False, showbackground=True, backgroundcolor='white'),
            zaxis=dict(showgrid=False, showbackground=True, backgroundcolor='white'),
        )
    )
    
    fig.show()

def plot3dDust(df, title, right):
    # Icosahedron neurotypical average
    if right == 1:
        shoulder_cols = ['ShoulderRight_X', 'ShoulderRight_Y', 'ShoulderRight_Z']
        wrist_cols = ['WristRight_X', 'WristRight_Y', 'WristRight_Z']
    else:
        shoulder_cols = ['ShoulderLeft_X', 'ShoulderLeft_Y', 'ShoulderLeft_Z']
        wrist_cols = ['WristLeft_X', 'WristLeft_Y', 'WristLeft_Z']
    
    
    center = df[shoulder_cols].loc[:100, :].mean()
    center.astype(np.int64)
    df = df.drop(df.index[-1])
    if right == 1:
        df['Wrist_X'] = df['WristRight_X'] - center['ShoulderRight_X']
        df['Wrist_Y'] = df['WristRight_Y'] - center['ShoulderRight_Y']
        df['Wrist_Z'] = df['WristRight_Z'] - center['ShoulderRight_Z']
        df['Hip_X'] = df['HipRight_X'] - center['ShoulderRight_X']
        df['Hip_Y'] = df['HipRight_Y'] - center['ShoulderRight_Y']
        df['Hip_Z'] = df['HipRight_Z'] - center['ShoulderRight_Z']
        df['Elbow_X'] = df['ElbowRight_X'] - center['ShoulderRight_X']
        df['Elbow_Y'] = df['ElbowRight_Y'] - center['ShoulderRight_Y']
        df['Elbow_Z'] = df['ElbowRight_Z'] - center['ShoulderRight_Z']
        df['Hand_X'] = df['HandRight_X'] - center['ShoulderRight_X']
        df['Hand_Y'] = df['HandRight_Y'] - center['ShoulderRight_Y']
        df['Hand_Z'] = df['HandRight_Z'] - center['ShoulderRight_Z']
        df['Shoulder_X'] = df['ShoulderRight_X'] - center['ShoulderRight_X']
        df['Shoulder_Y'] = df['ShoulderRight_Y'] - center['ShoulderRight_Y']
        df['Shoulder_Z'] = df['ShoulderRight_Z'] - center['ShoulderRight_Z']
        df['Thumb_X'] = df['ThumbRight_X'] - center['ShoulderRight_X']
        df['Thumb_Y'] = df['ThumbRight_Y'] - center['ShoulderRight_Y']
        df['Thumb_Z'] = df['ThumbRight_Z'] - center['ShoulderRight_Z']
        center['ShoulderRight_X'] -= center['ShoulderRight_X']
        center['ShoulderRight_Y'] -= center['ShoulderRight_Y']
        center['ShoulderRight_Z'] -= center['ShoulderRight_Z']
    else:
        df['Wrist_X'] = -(df['WristLeft_X'] - center['ShoulderLeft_X'])
        df['Wrist_Y'] = df['WristLeft_Y'] - center['ShoulderLeft_Y']
        df['Wrist_Z'] = df['WristLeft_Z'] - center['ShoulderLeft_Z']
        df['Hip_X'] = -(df['HipLeft_X'] - center['ShoulderLeft_X'])
        df['Hip_Y'] = df['HipLeft_Y'] - center['ShoulderLeft_Y']
        df['Hip_Z'] = df['HipLeft_Z'] - center['ShoulderLeft_Z']
        df['Elbow_X'] = -(df['ElbowLeft_X'] - center['ShoulderLeft_X'])
        df['Elbow_Y'] = df['ElbowLeft_Y'] - center['ShoulderLeft_Y']
        df['Elbow_Z'] = df['ElbowLeft_Z'] - center['ShoulderLeft_Z']
        df['Hand_X'] = -(df['HandLeft_X'] - center['ShoulderLeft_X'])
        df['Hand_Y'] = df['HandLeft_Y'] - center['ShoulderLeft_Y']
        df['Hand_Z'] = df['HandLeft_Z'] - center['ShoulderLeft_Z']
        df['Shoulder_X'] = -(df['ShoulderLeft_X'] - center['ShoulderLeft_X'])
        df['Shoulder_Y'] = df['ShoulderLeft_Y'] - center['ShoulderLeft_Y']
        df['Shoulder_Z'] = df['ShoulderLeft_Z'] - center['ShoulderLeft_Z']
        df['Thumb_X'] = -(df['ThumbLeft_X'] - center['ShoulderLeft_X'])
        df['Thumb_Y'] = df['ThumbLeft_Y'] - center['ShoulderLeft_Y']
        df['Thumb_Z'] = df['ThumbLeft_Z'] - center['ShoulderLeft_Z']
        center['ShoulderLeft_X'] -= center['ShoulderLeft_X']
        center['ShoulderLeft_Y'] -= center['ShoulderLeft_Y']
        center['ShoulderLeft_Z'] -= center['ShoulderLeft_Z']
        #call filter95 function to cut >95% percentile distances from shoulder

    fs = 30
    trim_for_abstract = fs * 120 #trim the first 2 minutes of the data
    df = df.loc[:trim_for_abstract, :]
    
    print(f'Filter95 Check {df.shape}')
    df_filtered = filterPositionData(df, center, right, subject)
    print('filter iqr+lp+smoothened shape')
    print(df_filtered.shape)
    
    # load the OBJ file and parse its contents - PERSON FIGURINE
    vertices = []
    faces = []
    with open('/Users/adithsrivatsa/Downloads/Man_Body_Valentino.obj', 'r') as file:
        for line in file:
            if line.startswith('v '):
                # parse vertex coordinates (x, y, z)
                vertex = list(map(float, line.strip().split()[1:]))
                temp = vertex[0]
                vertex[0] = (-1/600 * vertex[1]) + 0.1
                vertex[1] = (1/880 * vertex[2]) - 1.8
                vertex[2] = (-1/800 * temp) + 0.25
                vertices.append(vertex)
            elif line.startswith('f '):
                # parse face indices (assuming triangular faces)
                face_elements = line.strip().split()[1:]
                face_indices = []
                for element in face_elements:
                    # parse the vertex index (ignore texture and normal indices)
                    vertex_index = int(element.split('/')[0]) - 1
                    face_indices.append(vertex_index)
                faces.append(face_indices)
        
    #vertices and faces defined before 3D plots begin
    # convert vertices and faces to numpy arrays
    vertices = np.array(vertices)
    faces = np.array(faces)
    fig = go.Figure()
    fig = px.scatter_3d(df_filtered, x="Wrist_X", y="Wrist_Y", z="Wrist_Z")
    fig.update_traces(marker = dict(size = 2))

    # Create the figurine dude
    fig.add_trace(go.Mesh3d(
        x=vertices[:, 0],  # X coordinates of vertices
        y=vertices[:, 1],  # Y coordinates of vertices
        z=vertices[:, 2],  # Z coordinates of vertices
        i=faces[:, 0],  # Indices of first vertex of each face
        j=faces[:, 1],  # Indices of second vertex of each face
        k=faces[:, 2],  # Indices of third vertex of each face
        color='lightblue',  # Color of the figurine
        opacity=1,
        flatshading=True,
    ))

        # Update layout total
    fig.update_layout(
        title=title,
        scene=dict(
            aspectmode='data',
            xaxis=dict(title='X Axis'),
            yaxis=dict(title='Y Axis'),
            zaxis=dict(title='Z Axis')
        ),
        width=1000,
        height=1000,
        showlegend=False,
        margin=dict(l=50, r=50, b=50, t=50),
    )
        
    #clear out the background color
    fig.update_layout(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        plot_bgcolor='white'
    )
    # Show the figure
    fig.show()

In [ ]:
# %% Overall looping through each subject data

directory = '/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/'
#initialize data zones for icos
stats_icosahedron = pd.DataFrame(index=range(1, 41))
stats_forceavg_icosahedron = pd.DataFrame(index=range(1, 41))
stats_jointAngavg_icosahedron = pd.DataFrame(index=range(1, 41))
stats_synergy_icosahedron = pd.DataFrame(index=range(1, 41))
stats_vel_icos = pd.DataFrame(index=range(1, 41))
stats_acc_icos = pd.DataFrame(index=range(1, 41))
stats_mvt_icosahedron = pd.DataFrame(index=range(1,41))
stats_summary = pd.DataFrame(index=range(1,3))
zone_data = ['Zone ' + str(i) for i in range(1, 41)]
stats_icosahedron['Zones'] = zone_data
stats_vel_icos['Zones'] = zone_data
stats_acc_icos['Zones'] = zone_data
stats_forceavg_icosahedron['Zones'] = zone_data
stats_jointAngavg_icosahedron['Zones'] = zone_data
stats_synergy_icosahedron['Zones'] = zone_data
stats_mvt_icosahedron['Zones'] = zone_data
#THe fields for summary - Convhull, synergy correlations (not zone-based)
parameter_names = ['Convhull', 'Synergy']
stats_summary['Parameters'] = parameter_names
#initialize data zones for rect
stats_rectilinear = pd.DataFrame(index=range(1, 28))
stats_forceavg_rectilinear = pd.DataFrame(index=range(1, 28))
zone_data = ['Zone ' + str(i) for i in range(1, 28)]
stats_rectilinear['Zones'] = zone_data
stats_forceavg_rectilinear['Zones'] = zone_data

cumData = [[] for _ in range(3)]
rightDomList = ['s04', 's06', 's08', 's10', 's11', 's12', 's14', 'p01', 'p03', 'p10', 'p12', 'p13', 'p17', 'p33', 'p37']
right = 1
jointAngle_list = []

#parse through the directory, go through each csv file
for filename in sorted(os.listdir(directory)):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath) and not fnmatch.fnmatch(filename, '.DS_Store'):
        ind = filename.find('_')
        subject = filename[:ind]
        #Check if right or left to be analyzed
        #print(filename)
        print(subject)
        if subject in rightDomList:
            right = 1
        else:
            right = 0
        print(right)
        #print(filepath)
        df = pd.read_csv(filepath)
        #Call the big Function to assess percent Time spent in each zone
        shape_modeling(df, right, subject, jointAngle_list, stats_summary, stats_icosahedron, stats_rectilinear, stats_vel_icos, stats_acc_icos, stats_forceavg_icosahedron, stats_forceavg_rectilinear, stats_jointAngavg_icosahedron, stats_synergy_icosahedron, stats_mvt_icosahedron, cumData)

#define file names and path
directoryStats = '/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/StatisticsCollection/'
Stats_icos_filename = 'ZoneDistribution_Icosahedron_r2.xlsx'
Stats_rect_filename = 'ZoneDistribution_Rectilinear.xlsx'
fullStorage_icos_path = os.path.join(directoryStats, Stats_icos_filename)
fullStorage_rect_path = os.path.join(directoryStats, Stats_rect_filename)

#for vel and accel
stats_vel_icos_filename ='ZoneVelocity_Icosahedron.xlsx'
stats_accel_icos_filename = 'ZoneAcceleration_Icosahedron.xlsx'
fullStorage_vel_path = os.path.join(directoryStats, stats_vel_icos_filename)
fullStorage_accel_path = os.path.join(directoryStats, stats_accel_icos_filename)

#for forces
Stats_forces_icos_filename = 'ZoneForces_Icosahedron.xlsx'
Stats_forces_rect_filename = 'ZoneForces_Rectilinear.xlsx'
fullStorageForces_icos_path = os.path.join(directoryStats, Stats_forces_icos_filename)
fullStorageForces_rect_path = os.path.join(directoryStats, Stats_forces_rect_filename)

#for jointAngles
Stats_jointAngles_icos_filename = 'ZoneJointAngles_Icosahedron.xlsx'
fullStorageJA_icos_path = os.path.join(directoryStats, Stats_jointAngles_icos_filename)
Stats_synergy_icos_filename = 'ZoneSynergy_Icosahedron.xlsx'
fullStorageSynergy_icos_path = os.path.join(directoryStats, Stats_synergy_icos_filename)

#for movement
Stats_movement_icos_filename = 'ZoneMovement_Icosahedron.xlsx'
fullStorageMVT_icos_path = os.path.join(directoryStats, Stats_movement_icos_filename)

#for summer
stats_summary_filename = 'ParameterSummarybySubject.xlsx'
fullsummaryStorage_path = os.path.join(directoryStats, stats_summary_filename)

# write the stats_df to an Excel file
stats_icosahedron.to_excel(fullStorage_icos_path, engine='openpyxl', index=False)
stats_rectilinear.to_excel(fullStorage_rect_path, engine='openpyxl', index=False)
stats_forceavg_icosahedron.to_excel(fullStorageForces_icos_path, engine='openpyxl', index=False)
stats_forceavg_rectilinear.to_excel(fullStorageForces_rect_path, engine='openpyxl', index=False)
stats_jointAngavg_icosahedron.to_excel(fullStorageJA_icos_path, engine='openpyxl', index=False)
stats_synergy_icosahedron.to_excel(fullStorageSynergy_icos_path, engine='openpyxl', index=False)
stats_vel_icos.to_excel(fullStorage_vel_path, engine='openpyxl', index=False)
stats_acc_icos.to_excel(fullStorage_accel_path, engine='openpyxl', index=False)
stats_mvt_icosahedron.to_excel(fullStorageMVT_icos_path, engine='openpyxl', index=False)
stats_summary.to_excel(fullsummaryStorage_path, engine='openpyxl', index=False)

In [ ]:
# %% Analyze Wrist Position Data

#ICOSAHEDRON % TIME Wrist Position - Linear Regression
df_icosahedronSummary = pd.read_excel('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/StatisticsCollection/ZoneDistribution_Icosahedron_r2.xlsx')
df_rectilinearSummary = pd.read_excel('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/StatisticsCollection/ZoneDistribution_Rectilinear.xlsx')
clinical_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/phase1_clinicalscores.csv')
clinical_aratBreakdown = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase1-AllDataNoPHI_DATA_LABELS_2024-07-22_1711.csv')
directory = '/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/'
#parse through the directory, go through each csv file
fig100 = go.Figure()
figarat = go.Figure()
figGross = go.Figure()
figGrip = go.Figure()
figGraPinch = go.Figure()
figSynergy = go.Figure()
figOther = go.Figure()
color_count = 0
ttest_results_per_zone = []
for zone in range(40):
    percent_time = []
    arat_scores = []
    fmue_scores = []
    arat_Gross = []
    arat_Grip = []
    arat_GraPinch = []
    fmue_synergy = []
    zone_time_ss = []
    zone_time_nt = []
    for filename in sorted(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and not fnmatch.fnmatch(filename, '.DS_Store'):
            ind = filename.find('_')
            subject = filename[:ind]
            if subject[0] == 'p':
                fmue_curr = clinical_df[(clinical_df['subject'].str.lower() == subject) & (clinical_df['session'].str.lower() == 'baseline')]['fmue']
                arat_curr = clinical_df[(clinical_df['subject'].str.lower() == subject) & (clinical_df['session'].str.lower() == 'baseline')]['arat']
                ARATbreakdown_curr = clinical_aratBreakdown[(clinical_aratBreakdown['Record ID'].str.lower() == subject) & (clinical_aratBreakdown['Event Name'].str.lower() == 'baseline evals')]
                Synergylist_backup = clinical_aratBreakdown[(clinical_aratBreakdown['Record ID'].str.lower() == subject) & (clinical_aratBreakdown['Event Name'].str.lower() == 'screening')]
                Gross_curr = int(ARATbreakdown_curr['Gross Movement Total Score'])
                Grip_curr = int(ARATbreakdown_curr['Grip Subtest Total Score'])
                GraPinch_curr = [int(ARATbreakdown_curr['Grasp Subtest Total Score']), int(ARATbreakdown_curr['Pinch Subtest Total Score'])]
                GraPinch_curr = np.sum(GraPinch_curr)
                fmue_curr = int(fmue_curr)
                arat_curr = int(arat_curr)
                if ARATbreakdown_curr['Biceps- Flexor'].isna().any():
                    synergy_list = [int(Synergylist_backup['Biceps- Flexor']), int(Synergylist_backup['Triceps - Extensor']), int(Synergylist_backup['Elevation']), int(Synergylist_backup['Retraction']), int(Synergylist_backup['Abduction']), int(Synergylist_backup['External Rotation']), int(Synergylist_backup['Elbow Flexion']), int(Synergylist_backup['Supination']), int(Synergylist_backup['Abduction']), int(Synergylist_backup['Adduction/Internal Rotation']), int(Synergylist_backup['Elbow Extension']), int(Synergylist_backup['Pronation']), int(Synergylist_backup['Hand to Lumbar Spine']), int(Synergylist_backup['Shoulder flexion to 90 degrees with elbow at 0 degrees and forearm in neutral']), int(Synergylist_backup['Pronation/Supination with elbow at 90 degrees']), int(Synergylist_backup['Abduction to 90 degrees']), int(Synergylist_backup['Shoulder flexion 90 to 180 degrees with elbow at 0 degrees and forearm in neutral']), int(Synergylist_backup['Pronation/Supination with elbow at 0 degrees and shoulder flexed 30 to 90 degrees']), int(Synergylist_backup['Test deep tendon relfexes of the biceps OR long finger flexors, then triceps'])]
                else:
                    synergy_list = [int(ARATbreakdown_curr['Biceps- Flexor']), int(ARATbreakdown_curr['Triceps - Extensor']), int(ARATbreakdown_curr['Elevation']), int(ARATbreakdown_curr['Retraction']), int(ARATbreakdown_curr['Abduction']), int(ARATbreakdown_curr['External Rotation']), int(ARATbreakdown_curr['Elbow Flexion']), int(ARATbreakdown_curr['Supination']), int(ARATbreakdown_curr['Abduction']), int(ARATbreakdown_curr['Adduction/Internal Rotation']), int(ARATbreakdown_curr['Elbow Extension']), int(ARATbreakdown_curr['Pronation']), int(ARATbreakdown_curr['Hand to Lumbar Spine']), int(ARATbreakdown_curr['Shoulder flexion to 90 degrees with elbow at 0 degrees and forearm in neutral']), int(ARATbreakdown_curr['Pronation/Supination with elbow at 90 degrees']), int(ARATbreakdown_curr['Abduction to 90 degrees']), int(ARATbreakdown_curr['Shoulder flexion 90 to 180 degrees with elbow at 0 degrees and forearm in neutral']), int(ARATbreakdown_curr['Pronation/Supination with elbow at 0 degrees and shoulder flexed 30 to 90 degrees']), int(ARATbreakdown_curr['Test deep tendon relfexes of the biceps OR long finger flexors, then triceps'])]
                synergy_curr = np.sum(synergy_list)
                zone_time_ss.append(df_icosahedronSummary[subject][zone])
            else:
                fmue_curr = 66
                arat_curr = 57
                Gross_curr = 9
                Grip_curr = 12
                GraPinch_curr = 36
                synergy_curr = 36
                zone_time_nt.append(df_icosahedronSummary[subject][zone])
            fmue_scores.append(fmue_curr)
            arat_scores.append(arat_curr)
            arat_Gross.append(Gross_curr)
            arat_Grip.append(Grip_curr)
            arat_GraPinch.append(GraPinch_curr)
            fmue_synergy.append(synergy_curr)
            #Find the percenttime in this zone
            percent_curr = df_icosahedronSummary[subject][zone]
            percent_time.append(percent_curr)
    #color='rgb({}, {}, {})'.format(np.random.randint(0, 192), np.random.randint(0, 192), np.random.randint(0, 192))
    colors = ['#33a02c', '#ff7f00', '#6a3d9a', '#b15928', '#e377c2', '#bcbd22', '#17becf', '#FF0000', '#000000', '#7f7f7f']
    #print('fmue_scores', fmue_scores)
    #print('percent time', percent_time)
    
    percent_time = np.array(percent_time)
    fmue_scores = np.array(fmue_scores)
    arat_scores = np.array(arat_scores)
    arat_Gross = np.array(arat_Gross)
    arat_Grip = np.array(arat_Grip)
    arat_GraPinch = np.array(arat_GraPinch)
    fmue_synergy = np.array(fmue_synergy)
    fmue_other = fmue_scores - fmue_synergy
    zone_time_ss = np.array(zone_time_ss)
    zone_time_nt = np.array(zone_time_nt)
    print(fmue_synergy)

    linearRegression(fmue_scores, percent_time, zone, fig100, '% Time Icosahedron v FMUE', color_count)
    linearRegression(arat_scores, percent_time, zone, figarat, '% Time Icosahedron v ARAT', color_count)
    linearRegression(arat_Gross, percent_time, zone, figGross, '% Time Icos v Gross Arat Subscore', color_count)
    linearRegression(arat_Grip, percent_time, zone, figGrip, '% Time Icos v Grip Arat Subscore', color_count)
    linearRegression(arat_GraPinch, percent_time, zone, figGraPinch, '% Time Icos v GraspPinch Arat Subscore', color_count)
    linearRegression(fmue_synergy, percent_time, zone, figSynergy, '% Time Icos v Synergy FMUE Subscore', color_count)            
    linearRegression(fmue_other, percent_time, zone, figOther, '% Time Icos v Gross FMUE Subscore', color_count)   
    color_count +=1

    t_stat, p_value = ttest_ind(zone_time_nt, zone_time_ss, equal_var=False)
    if p_value < 0.05:
        ttest_results_per_zone.append({'Zone': zone+1, 'p_val': p_value, 'Mean Stroke': np.mean(zone_time_ss), 'Mean NT': np.mean(zone_time_nt)})

# add legend
fig100.update_layout(
    legend=dict(
        #orientation="h",
        yanchor="top",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=800,
    height=600,
    xaxis_title='FMUE Scores',
    yaxis_title='Percent Time Spent in Zone (%)',
    title='Icosahedron Regression'
)
fig100.show()
figarat.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Regression')
figarat.show()
figGross.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Gross Subscore Regression')
figGross.show()
figGrip.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Grip Subscore Regression')
figGrip.show()
figGraPinch.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Grasp/Pinch Subscore Regression')
figGraPinch.show()
figSynergy.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='FMUE synergy Subscore Regression')
figSynergy.show()
figOther.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='FMUE Gross Subscore Regression')
figOther.show()


#add a super small constant to 0s to avoid log issues
tiny_constant = 1e-6

# Check the variability per Icosahedron zone of the neurotypicals
s_columns = [col for col in df_icosahedronSummary.columns if col.startswith('s')]
p_columns = [col for col in df_icosahedronSummary.columns if col.startswith('p')]

#print(s_columns)

# Create a DataFrame to store CI results
ci_results = []

# Iterate over each row to calculate confidence intervals
for index, row in df_icosahedronSummary.iterrows():
    s_values = row[s_columns].dropna()
    s_values = s_values / 100 #switch to probability from percentage
    mean = np.mean(s_values)
    sem = stats.sem(s_values)
    ci = stats.t.interval(0.95, len(s_values)-1, loc=mean, scale=sem)
    
    ci_lower = max(ci[0], tiny_constant)
    
    ci_results.append({
        'Zone': f'Zone {index+1}',
        'CI Lower': ci_lower,
        'CI Upper': ci[1],
        'Mean': mean
    })
    
#print(ci_results)

# Initialize a dictionary to store the count of p_columns outside CI
ci_df = pd.DataFrame(ci_results)
outside_counts = {p_col: 0 for p_col in p_columns}
outsideCI_perzone = {zone: 0 for zone in range(len(df_icosahedronSummary))}
outsideCI_direction = {zone: {p_col: None for p_col in p_columns} for zone in range(len(df_icosahedronSummary))}


# Check if p_column values are outside CI
for index, row in df_icosahedronSummary.iterrows():
    zone = f'Zone {index + 1}'
    ci_row= ci_df[ci_df['Zone'] == zone].iloc[0]
    
    ci_lower = ci_row['CI Lower']
    ci_upper = ci_row['CI Upper']
    
    if not pd.isna(ci_lower) and not pd.isna(ci_upper):
        for p_col in p_columns:
            p_value = row[p_col] / 100 + tiny_constant
            if p_value < ci_lower:
                # Value is below the lower CI bound
                outside_counts[p_col] += 1
                outsideCI_perzone[index] += 1
                outsideCI_direction[index][p_col] = 'Lower'
            
            elif p_value > ci_upper:
                # Value is above the upper CI bound
                outside_counts[p_col] += 1
                outsideCI_perzone[index] += 1
                outsideCI_direction[index][p_col] = 'Higher'
            
            else:
                # Value is within the CI bounds
                outsideCI_direction[index][p_col] = 'Within'
                #print(f'{p_col}: {p_value} is outside {ci_lower} to {ci_upper}')
nt_ci_data = ci_results
print(f'Plotting icosahedron based on OutsideCIPerZone {outsideCI_perzone}')

counts_icos = []
# Print the counts of p_columns outside CI
for p_col, count in outside_counts.items():
    print(f'{p_col} is outside the confidence intervals in {count} zones.')
    counts_icos.append(count)


colors = sns.color_palette("husl", len(p_columns))  # Husl generates a distinct color palette
p_color_map = {col: colors[i] for i, col in enumerate(p_columns)}

plot_data = pd.DataFrame()
for index, zone in df_icosahedronSummary.iterrows():
    s_zone_data = zone[s_columns].dropna() /100 #convert back to probability instead of %
    p_zone_data = zone[p_columns].dropna() / 100
    
    #add the tiny constant to the values
    s_zone_data = s_zone_data.astype('float') + tiny_constant
    p_zone_data = p_zone_data.astype('float') + tiny_constant

    s_temp_df = pd.DataFrame({
        'Values': s_zone_data.values,
        'Zone': zone['Zones'] if 'Zone' in df_icosahedronSummary.columns else f'{index+1}',
        'Type': 's',
        'Color': 'black',
        'Subject': 'Neurotypicals'
    })
    
    p_temp_df = pd.DataFrame({
        'Values': p_zone_data.values,
        'Zone': zone['Zones'] if 'Zone' in df_icosahedronSummary.columns else f'{index+1}',
        'Type': 'p',
        'Color': [p_color_map[col] for col in p_zone_data.index], #assign colors
        'Subject': p_zone_data.index
    })
    
    plot_data = pd.concat([plot_data, s_temp_df, p_temp_df], axis=0)
    
#plot box and whisker plot
plt.figure(figsize=(12,8), dpi=600)
stripplot = sns.stripplot(x='Zone', y='Values', data=plot_data, hue='Subject', palette={'Neurotypicals': 'black', **p_color_map}, size=5, jitter=True, dodge=True)
s_coords = stripplot.collections[0].get_offsets()[:, 0]
for idx, ci in ci_df.iterrows():
    #x_coord = s_coords[idx]
    plt.scatter(idx-0.4, ci['Mean'], edgecolor='black', facecolor='white', s=20, zorder=5)
    print([ci['CI Lower'], ci['CI Upper']])
    plt.errorbar(x=idx-0.4, y=ci['Mean'], yerr=[[ci['Mean'] - ci['CI Lower']], [ci['CI Upper'] - ci['Mean']]],
                 fmt='o', color='black', capsize=5)

plt.title(f'Wrist Position Probability in Icosahedron-based Zones')  # Assuming there's a column named 'Area'
plt.xlabel('Zones (1-40)')
plt.ylabel('Probability ')
plt.yscale('log')
plt.ylim(8e-7, 1e0) #ignore the 0s
plt.legend(title='Subject', loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0.)

# Setting black outlines for boxes and whiskers
#for patch in plt.gca().artists:
 #   patch.set_edgecolor('black')
  #  patch.set_linewidth(1.5)

# Display plot
plt.show()

#Cumulative distribution for icosahedron
mean_values = ci_df[['Zone', 'Mean']].copy() #store the mean_values of the nt group
zones_sorted = mean_values.sort_values(by='Mean', ascending=False)
zones_sorted['Cumulative Sum'] = zones_sorted['Mean'].cumsum()

uniform_zones = np.linspace(0.05, 1, len(zones_sorted))

# Step 4: Plot the cumulative distribution
plt.figure(figsize=(10, 6), dpi=600)
sns.lineplot(x=zones_sorted['Zone'], y=zones_sorted['Cumulative Sum'], color='black', label = 'Neurotypical')
sns.lineplot(x=zones_sorted['Zone'], y=uniform_zones, marker='o', color='brown', linestyle='--', label='Uniform')

# Plot cumulative distribution for s_columns
#for s_col in s_columns:
 #   s_sorted = df_icosahedronSummary[s_col].dropna().sort_values(ascending=False)
  #  s_cumsum = s_sorted.cumsum() / 100
   # sns.lineplot(x=range(len(s_sorted)), y=s_cumsum, marker='o', label=f'Cumulative Sum ({s_col})', color='black')


# Step 4: Calculate cumulative sums for p_columns based on the sorted order of s_columns
for p_col in p_columns:
    #p_sorted = df_icosahedronSummary[p_col].dropna().sort_values(ascending=False)
    #p_cumsum = p_sorted.cumsum()/ 100
    p_data_sorted = df_icosahedronSummary.loc[zones_sorted.index, p_col].dropna().cumsum() / 100
    sns.lineplot(x=zones_sorted['Zone'], y=p_data_sorted.values, marker='o', linestyle='--', label=f'({p_col})', color=p_color_map[p_col])
    #sns.lineplot(x=range(len(p_sorted)), y=p_cumsum, marker='o', label=f'{p_col}', color=p_color_map[p_col])
# Customizing the plot
plt.title('Cumulative Probability Distribution by Icosahedron Zone')
plt.xlabel('Zones Ordered by Tendency')
plt.ylabel('Cumulative Probability')
plt.xticks(rotation=90)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

#Create a grid - heatmap to show the different plots
zones = list(range(1, len(mean_values)+1))  # 40 zones

# Prepare an empty 20x20 grid
heatmap_data = np.zeros((20, len(mean_values)))  # 20 individuals, 20 zones
heatmap_data[0, 0:len(mean_values)] = mean_values['Mean'] - mean_values['Mean']

# Create a DataFrame with the same structure
for index, zone in df_icosahedronSummary.iterrows():
    p_zone_data = zone[p_columns].dropna() / 100  # Convert back to probability instead of %
    
    # Subtract the mean from each individual's data
    mean_value = mean_values['Mean'][index]  # Get the mean value for this zone
    heatmap_data[1:, index] = p_zone_data - mean_value  # Store the difference

icos_strk_data = p_zone_data
# Plot the heatmap
plt.figure(figsize=(10, 8), dpi=300)

# Option 1 - vertical layering
order = [8, 2, 5, 3, 4, 1, 7, 6, 19, 17, 12, 9, 15, 14, 16, 10, 13, 11, 18, 20, 28, 22, 25, 23, 24, 21, 27, 26, 39, 37, 32, 29, 35, 34, 36, 30, 33, 31, 38, 40]

#option 2 - horizontal layering
order = [5, 3, 4, 17, 12, 11, 13, 18, 2, 1, 7, 19, 9, 10, 20, 8, 6, 15, 14, 16]

# option 3 - poster-> anterior hamiltonian
order = [8, 2, 5, 3, 4, 1, 6, 7, 19, 17, 12, 11, 13, 10, 9, 15, 14, 16, 20, 18, 28, 22, 25, 23, 24, 21, 26, 27, 39, 37, 32, 31, 33, 30, 29, 35, 34, 36, 40, 38]
order_cols = np.array(order) - 1

#organize the subjects by FMUE score
index_nt = np.where(fmue_scores == 66)[0][0]
sorted_subj = np.argsort(-fmue_scores[:index_nt])
print(f"Sorted order of subjects: {sorted_subj}")
sorted_idx = np.array(sorted_subj) + 1
order_rows = np.insert(sorted_idx, 0, 0)

heatmap_temp = heatmap_data[order_rows, :]
heatmap_plot = heatmap_temp[:, order_cols]

# Create a custom colormap based on 'coolwarm' but with white at the center (0)
coolwarm_custom = LinearSegmentedColormap.from_list('coolwarm_custom', ['blue', 'white', 'red'])

# Create a color map where negative values are blue, positive are red, and zero is white
sns.heatmap(heatmap_plot, cmap=coolwarm_custom, center=0, annot=False, linewidths=0.5, xticklabels=range(1, 41))

# Set plot titles and labels
plt.title('Heatmap of Upper Extremity Movements (Difference from Mean Neurotypical)')
plt.xlabel('Zones (1-40)')
plt.ylabel('Individuals (1-20)')

# Show the plot
plt.show()

#calculate overall stroke group difference array
heatDifference_icosahedron = np.mean(heatmap_temp[-19:, :], axis=0)


#Weakness identification
ci_results

# RECTILINEAR NOW

#ICOSAHEDRON % TIME Wrist Position - Linear Regression
df_rectilinearSummary = pd.read_excel('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/StatisticsCollection/ZoneDistribution_Rectilinear.xlsx')
clinical_df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/phase1_clinicalscores.csv')
clinical_aratBreakdown = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/ClinicalFiles/ExoNETPhase1-AllDataNoPHI_DATA_LABELS_2024-07-22_1711.csv')
directory = '/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/'
#parse through the directory, go through each csv file
fig100 = go.Figure()
figarat = go.Figure()
figGross = go.Figure()
figGrip = go.Figure()
figGraPinch = go.Figure()
figSynergy = go.Figure()
figOther = go.Figure()
color_count = 0
ttest_results_per_zone = []
for zone in range(27):
    percent_time = []
    arat_scores = []
    fmue_scores = []
    arat_Gross = []
    arat_Grip = []
    arat_GraPinch = []
    fmue_synergy = []
    zone_time_ss = []
    zone_time_nt = []
    for filename in sorted(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and not fnmatch.fnmatch(filename, '.DS_Store'):
            ind = filename.find('_')
            subject = filename[:ind]
            if subject[0] == 'p':
                fmue_curr = clinical_df[(clinical_df['subject'].str.lower() == subject) & (clinical_df['session'].str.lower() == 'baseline')]['fmue']
                arat_curr = clinical_df[(clinical_df['subject'].str.lower() == subject) & (clinical_df['session'].str.lower() == 'baseline')]['arat']
                ARATbreakdown_curr = clinical_aratBreakdown[(clinical_aratBreakdown['Record ID'].str.lower() == subject) & (clinical_aratBreakdown['Event Name'].str.lower() == 'baseline evals')]
                Synergylist_backup = clinical_aratBreakdown[(clinical_aratBreakdown['Record ID'].str.lower() == subject) & (clinical_aratBreakdown['Event Name'].str.lower() == 'screening')]
                Gross_curr = int(ARATbreakdown_curr['Gross Movement Total Score'])
                Grip_curr = int(ARATbreakdown_curr['Grip Subtest Total Score'])
                GraPinch_curr = [int(ARATbreakdown_curr['Grasp Subtest Total Score']), int(ARATbreakdown_curr['Pinch Subtest Total Score'])]
                GraPinch_curr = np.sum(GraPinch_curr)
                fmue_curr = int(fmue_curr)
                arat_curr = int(arat_curr)
                if ARATbreakdown_curr['Biceps- Flexor'].isna().any():
                    synergy_list = [int(Synergylist_backup['Biceps- Flexor']), int(Synergylist_backup['Triceps - Extensor']), int(Synergylist_backup['Elevation']), int(Synergylist_backup['Retraction']), int(Synergylist_backup['Abduction']), int(Synergylist_backup['External Rotation']), int(Synergylist_backup['Elbow Flexion']), int(Synergylist_backup['Supination']), int(Synergylist_backup['Abduction']), int(Synergylist_backup['Adduction/Internal Rotation']), int(Synergylist_backup['Elbow Extension']), int(Synergylist_backup['Pronation']), int(Synergylist_backup['Hand to Lumbar Spine']), int(Synergylist_backup['Shoulder flexion to 90 degrees with elbow at 0 degrees and forearm in neutral']), int(Synergylist_backup['Pronation/Supination with elbow at 90 degrees']), int(Synergylist_backup['Abduction to 90 degrees']), int(Synergylist_backup['Shoulder flexion 90 to 180 degrees with elbow at 0 degrees and forearm in neutral']), int(Synergylist_backup['Pronation/Supination with elbow at 0 degrees and shoulder flexed 30 to 90 degrees']), int(Synergylist_backup['Test deep tendon relfexes of the biceps OR long finger flexors, then triceps'])]
                else:
                    synergy_list = [int(ARATbreakdown_curr['Biceps- Flexor']), int(ARATbreakdown_curr['Triceps - Extensor']), int(ARATbreakdown_curr['Elevation']), int(ARATbreakdown_curr['Retraction']), int(ARATbreakdown_curr['Abduction']), int(ARATbreakdown_curr['External Rotation']), int(ARATbreakdown_curr['Elbow Flexion']), int(ARATbreakdown_curr['Supination']), int(ARATbreakdown_curr['Abduction']), int(ARATbreakdown_curr['Adduction/Internal Rotation']), int(ARATbreakdown_curr['Elbow Extension']), int(ARATbreakdown_curr['Pronation']), int(ARATbreakdown_curr['Hand to Lumbar Spine']), int(ARATbreakdown_curr['Shoulder flexion to 90 degrees with elbow at 0 degrees and forearm in neutral']), int(ARATbreakdown_curr['Pronation/Supination with elbow at 90 degrees']), int(ARATbreakdown_curr['Abduction to 90 degrees']), int(ARATbreakdown_curr['Shoulder flexion 90 to 180 degrees with elbow at 0 degrees and forearm in neutral']), int(ARATbreakdown_curr['Pronation/Supination with elbow at 0 degrees and shoulder flexed 30 to 90 degrees']), int(ARATbreakdown_curr['Test deep tendon relfexes of the biceps OR long finger flexors, then triceps'])]
                synergy_curr = np.sum(synergy_list)
                zone_time_ss.append(df_rectilinearSummary[subject][zone])
            else:
                fmue_curr = 66
                arat_curr = 57
                Gross_curr = 9
                Grip_curr = 12
                GraPinch_curr = 36
                synergy_curr = 36
                zone_time_nt.append(df_icosahedronSummary[subject][zone])
            fmue_scores.append(fmue_curr)
            arat_scores.append(arat_curr)
            arat_Gross.append(Gross_curr)
            arat_Grip.append(Grip_curr)
            arat_GraPinch.append(GraPinch_curr)
            fmue_synergy.append(synergy_curr)
            #Find the percenttime in this zone
            percent_curr = df_rectilinearSummary[subject][zone]
            percent_time.append(percent_curr)
    #color='rgb({}, {}, {})'.format(np.random.randint(0, 192), np.random.randint(0, 192), np.random.randint(0, 192))
    colors = ['#33a02c', '#ff7f00', '#6a3d9a', '#b15928', '#e377c2', '#bcbd22', '#17becf', '#FF0000', '#000000', '#7f7f7f']
    #print('fmue_scores', fmue_scores)
    #print('percent time', percent_time)
    
    percent_time = np.array(percent_time)
    fmue_scores = np.array(fmue_scores)
    arat_scores = np.array(arat_scores)
    arat_Gross = np.array(arat_Gross)
    arat_Grip = np.array(arat_Grip)
    arat_GraPinch = np.array(arat_GraPinch)
    fmue_synergy = np.array(fmue_synergy)
    fmue_other = fmue_scores - fmue_synergy
    zone_time_ss = np.array(zone_time_ss)
    zone_time_nt = np.array(zone_time_nt)
    print(fmue_synergy)

    linearRegression(fmue_scores, percent_time, zone, fig100, '% Time Rectilinear v FMUE', color_count)
    linearRegression(arat_scores, percent_time, zone, figarat, '% Time Rectilinear v ARAT', color_count)
    linearRegression(arat_Gross, percent_time, zone, figGross, '% Time Rectilinear v Gross Arat Subscore', color_count)
    linearRegression(arat_Grip, percent_time, zone, figGrip, '% Time Rectilinear v Grip Arat Subscore', color_count)
    linearRegression(arat_GraPinch, percent_time, zone, figGraPinch, '% Time Rectilinear v GraspPinch Arat Subscore', color_count)
    linearRegression(fmue_synergy, percent_time, zone, figSynergy, '% Time Rectilinear v Synergy FMUE Subscore', color_count)            
    linearRegression(fmue_other, percent_time, zone, figOther, '% Time Rectilinear v Gross FMUE Subscore', color_count)   
    color_count +=1

    t_stat, p_value = ttest_ind(zone_time_nt, zone_time_ss, equal_var=False)
    if p_value < 0.05:
        ttest_results_per_zone.append({'Zone': zone+1, 'p_val': p_value, 'Mean Stroke': np.mean(zone_time_ss), 'Mean NT': np.mean(zone_time_nt)})

# add legend
fig100.update_layout(
    legend=dict(
        #orientation="h",
        yanchor="top",
        y=1.02,
        xanchor="right",
        x=1
    ),
    width=800,
    height=600,
    xaxis_title='FMUE Scores',
    yaxis_title='Percent Time Spent in Zone (%)',
    title='Rectilinear Regression'
)
fig100.show()
figarat.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Regression')
figarat.show()
figGross.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Gross Subscore Regression')
figGross.show()
figGrip.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Grip Subscore Regression')
figGrip.show()
figGraPinch.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='Arat Grasp/Pinch Subscore Regression')
figGraPinch.show()
figSynergy.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='FMUE synergy Subscore Regression')
figSynergy.show()
figOther.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), title='FMUE Gross Subscore Regression')
figOther.show()

# Check the variability per Rectilinear zone of the neurotypicals
s_columns = [col for col in df_rectilinearSummary.columns if col.startswith('s')]
p_columns = [col for col in df_rectilinearSummary.columns if col.startswith('p')]


# Create a DataFrame to store CI results
ci_results = []

# Iterate over each row to calculate confidence intervals
for index, row in df_rectilinearSummary.iterrows():
    s_values = row[s_columns].dropna()
    s_values = s_values / 100
    mean = np.mean(s_values)
    sem = stats.sem(s_values)
    ci = stats.t.interval(0.95, len(s_values)-1, loc=mean, scale=sem)
    
    ci_lower = max(ci[0], tiny_constant)
    
    ci_results.append({
        'Zone': f'Zone {index+1}',
        'CI Lower': ci_lower,
        'CI Upper': ci[1],
        'Mean': mean
    })
    
print(ci_results)

# Initialize a dictionary to store the count of p_columns outside CI
ci_df = pd.DataFrame(ci_results)
Rect_outside_counts = {p_col: 0 for p_col in p_columns}
Rect_outsideCI_perzone = {zone: 0 for zone in range(len(df_rectilinearSummary))}
Rect_outsideCI_direction = {zone: {p_col: None for p_col in p_columns} for zone in range(len(df_rectilinearSummary))}


# Check if p_column values are outside CI
for index, row in df_rectilinearSummary.iterrows():
    zone = f'Zone {index + 1}'
    ci_row= ci_df[ci_df['Zone'] == zone].iloc[0]
    
    ci_lower = ci_row['CI Lower']
    ci_upper = ci_row['CI Upper']
    
    if not pd.isna(ci_lower) and not pd.isna(ci_upper):
        for p_col in p_columns:
            p_value = row[p_col] / 100 + tiny_constant
            if p_value < ci_lower:
                # Value is below the lower CI bound
                Rect_outside_counts[p_col] += 1
                Rect_outsideCI_perzone[index] += 1
                Rect_outsideCI_direction[index][p_col] = 'Lower'
            
            elif p_value > ci_upper:
                # Value is above the upper CI bound
                Rect_outside_counts[p_col] += 1
                Rect_outsideCI_perzone[index] += 1
                Rect_outsideCI_direction[index][p_col] = 'Higher'
            
            else:
                # Value is within the CI bounds
                Rect_outsideCI_direction[index][p_col] = 'Within'
                #print(f'{p_col}: {p_value} is outside {ci_lower} to {ci_upper}')
Rect_nt_ci_data = ci_results
print(f'Plotting Rectilinear based on OutsideCIPerZone {Rect_outsideCI_perzone}')

# Print the counts of p_columns outside CI
counts_rect = []
for p_col, count in outside_counts.items():
    print(f'{p_col} is outside the confidence intervals in {count} zones.')
    counts_rect.append(count)

colors = sns.color_palette("husl", len(p_columns))  # Husl generates a distinct color palette
p_color_map = {col: colors[i] for i, col in enumerate(p_columns)}

plot_data = pd.DataFrame()
for index, zone in df_rectilinearSummary.iterrows():
    s_zone_data = zone[s_columns].dropna() / 100 # convert back to probability instead of %
    p_zone_data = zone[p_columns].dropna() / 100
    
    #add the tiny constant to the values
    s_zone_data = s_zone_data.astype('float') + tiny_constant
    p_zone_data = p_zone_data.astype('float') + tiny_constant
    
    s_temp_df = pd.DataFrame({
        'Values': s_zone_data.values,
        'Zone': zone['Zones'] if 'Zone' in df_rectilinearSummary.columns else f'{index+1}',
        'Type': 's',
        'Color': 'black',
        'Subject': 'Neurotypicals'
    })
    
    p_temp_df = pd.DataFrame({
        'Values': p_zone_data.values,
        'Zone': zone['Zones'] if 'Zone' in df_rectilinearSummary.columns else f'{index+1}',
        'Type': 'p',
        'Color': [p_color_map[col] for col in p_zone_data.index], #assign colors
        'Subject': p_zone_data.index
    })
    
    plot_data = pd.concat([plot_data, s_temp_df, p_temp_df], axis=0)

# Create a DataFrame that has the diff of stroke survivors to the mean values for each zone
stroke_df = df_rectilinearSummary[p_columns]
stroke_diff_df = stroke_df.copy()
print(stroke_df)
for i in range(27):
    mean_val = Rect_nt_ci_data[i]['Mean']  # Assuming Rect_nt_ci_data is a DataFrame or Series
    print(mean_val)
    stroke_diff_df.iloc[i] = (stroke_df.iloc[i] / 100).subtract(mean_val, axis=0)

#plot box and whisker plot
plt.figure(figsize=(12,8), dpi=600)
stripplot = sns.stripplot(x='Zone', y='Values', data=plot_data, hue='Subject', palette={'Neurotypicals': 'black', **p_color_map}, size=5, jitter=True, dodge=True)
s_coords = stripplot.collections[0].get_offsets()[:, 0]
for idx, ci in ci_df.iterrows():
    #x_coord = s_coords[idx]
    plt.scatter(idx-0.4, ci['Mean'], edgecolor='black', facecolor='white', s=20, zorder=5)
    plt.errorbar(x=idx-0.4, y=ci['Mean'], yerr=[[ci['Mean'] - ci['CI Lower']], [ci['CI Upper'] - ci['Mean']]],
                 fmt='o', color='black', capsize=5)



plt.title(f'Wrist Position Probability in Rectilinear-based Zones')  # Assuming there's a column named 'Area'
plt.xlabel('Zones (1-27)')
plt.ylabel('Probability')
plt.yscale('log')
plt.ylim(8e-7, 1e0) #ignore the 0s
plt.legend(title='Subject', loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0.)

# Setting black outlines for boxes and whiskers
#for patch in plt.gca().artists:
 #   patch.set_edgecolor('black')
  #  patch.set_linewidth(1.5)

# Display plot
plt.show()

#plot the cumulative distribution curve for the NT mean in black
# stroke can have the same colors but lines 
mean_values = ci_df[['Zone', 'Mean']].copy()
zones_sorted = mean_values.sort_values(by='Mean', ascending=False)
zones_sorted['Cumulative Sum'] = zones_sorted['Mean'].cumsum()

uniform_zones = np.linspace(0.05, 1, len(zones_sorted))

# Step 4: Plot the cumulative distribution
plt.figure(figsize=(10, 6), dpi=600)
sns.lineplot(x=zones_sorted['Zone'], y=zones_sorted['Cumulative Sum'], marker='o', color='black', label = 'Neurotypical')
sns.lineplot(x=zones_sorted['Zone'], y=uniform_zones, color='brown', linestyle='--', label='Uniform')

# Step 4: Calculate cumulative sums for p_columns based on the sorted order of s_columns
for p_col in p_columns:
    #p_sorted = df_rectilinearSummary[p_col].dropna().sort_values(ascending=False)
    #p_cumsum = p_sorted.cumsum()/ 100
    p_data_sorted = df_rectilinearSummary.loc[zones_sorted.index, p_col].dropna().cumsum() / 100
    sns.lineplot(x=zones_sorted['Zone'], y=p_data_sorted.values, marker='o', linestyle='--', label=f'({p_col})', color=p_color_map[p_col])
    #sns.lineplot(x=range(len(p_sorted)), y=p_cumsum, marker='o', label=f'{p_col}', color=p_color_map[p_col])
# Customizing the plot
plt.title('Cumulative Probability Distribution by Rectilinear Zone')
plt.xlabel('Zones Ordered by Tendency')
plt.ylabel('Cumulative Probability')
plt.xticks(rotation=90)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

#3D plots
df = pd.read_csv('/Users/adithsrivatsa/Desktop/SphericalDistributionAnalysis/s06_freexp_control.csv')

subject = 's06'
rightDomList = ['s04', 's06', 's08', 's10', 's11', 's12', 's14', 'p01', 'p03', 'p10', 'p12', 'p13', 'p17', 'p33', 'p37']
right = 1

if subject in rightDomList:
    right = 1
else:
    right = 0
    
if right == 1: 
    shoulder_cols = ['ShoulderRight_X', 'ShoulderRight_Y', 'ShoulderRight_Z']
    wrist_cols = ['WristRight_X', 'WristRight_Y', 'WristRight_Z']
else:
    shoulder_cols = ['ShoulderLeft_X', 'ShoulderLeft_Y', 'ShoulderLeft_Z']
    wrist_cols = ['WristLeft_X', 'WristLeft_Y', 'WristLeft_Z']

#Normalize to shoulder
center = df[shoulder_cols].loc[:100, :].mean()
center.astype(np.int64)
df.dropna(subset = wrist_cols)
if right == 1:
    df['Wrist_X'] = df['WristRight_X'] - center['ShoulderRight_X']
    df['Wrist_Y'] = df['WristRight_Y'] - center['ShoulderRight_Y']
    df['Wrist_Z'] = df['WristRight_Z'] - center['ShoulderRight_Z']
    df['Hip_X'] = df['HipRight_X'] - center['ShoulderRight_X']
    df['Hip_Y'] = df['HipRight_Y'] - center['ShoulderRight_Y']
    df['Hip_Z'] = df['HipRight_Z'] - center['ShoulderRight_Z']
    df['Elbow_X'] = df['ElbowRight_X'] - center['ShoulderRight_X']
    df['Elbow_Y'] = df['ElbowRight_Y'] - center['ShoulderRight_Y']
    df['Elbow_Z'] = df['ElbowRight_Z'] - center['ShoulderRight_Z']
    df['Hand_X'] = df['HandRight_X'] - center['ShoulderRight_X']
    df['Hand_Y'] = df['HandRight_Y'] - center['ShoulderRight_Y']
    df['Hand_Z'] = df['HandRight_Z'] - center['ShoulderRight_Z']
    df['Shoulder_X'] = df['ShoulderRight_X'] - center['ShoulderRight_X']
    df['Shoulder_Y'] = df['ShoulderRight_Y'] - center['ShoulderRight_Y']
    df['Shoulder_Z'] = df['ShoulderRight_Z'] - center['ShoulderRight_Z']
    df['Thumb_X'] = df['ThumbRight_X'] - center['ShoulderRight_X']
    df['Thumb_Y'] = df['ThumbRight_Y'] - center['ShoulderRight_Y']
    df['Thumb_Z'] = df['ThumbRight_Z'] - center['ShoulderRight_Z']
    df['ShoulderRef_X'] = df['Shoulder_X']
    df['ShoulderRef_Y'] = df['Shoulder_Y']
    df['ShoulderRef_Z'] = df['Shoulder_Z']
    df['ShoulderOther_X'] = df['ShoulderLeft_X']
    df['ShoulderOther_Y'] = df['ShoulderLeft_Y']
    df['ShoulderOther_Z'] = df['ShoulderLeft_Z']
    center['ShoulderRight_X'] -= center['ShoulderRight_X']
    center['ShoulderRight_Y'] -= center['ShoulderRight_Y']
    center['ShoulderRight_Z'] -= center['ShoulderRight_Z']
else:
    df['Wrist_X'] = -(df['WristLeft_X'] - center['ShoulderLeft_X'])
    df['Wrist_Y'] = df['WristLeft_Y'] - center['ShoulderLeft_Y']
    df['Wrist_Z'] = df['WristLeft_Z'] - center['ShoulderLeft_Z']
    df['Hip_X'] = -(df['HipLeft_X'] - center['ShoulderLeft_X'])
    df['Hip_Y'] = df['HipLeft_Y'] - center['ShoulderLeft_Y']
    df['Hip_Z'] = df['HipLeft_Z'] - center['ShoulderLeft_Z']
    df['Elbow_X'] = -(df['ElbowLeft_X'] - center['ShoulderLeft_X'])
    df['Elbow_Y'] = df['ElbowLeft_Y'] - center['ShoulderLeft_Y']
    df['Elbow_Z'] = df['ElbowLeft_Z'] - center['ShoulderLeft_Z']
    df['Hand_X'] = -(df['HandLeft_X'] - center['ShoulderLeft_X'])
    df['Hand_Y'] = df['HandLeft_Y'] - center['ShoulderLeft_Y']
    df['Hand_Z'] = df['HandLeft_Z'] - center['ShoulderLeft_Z']
    df['Shoulder_X'] = -(df['ShoulderLeft_X'] - center['ShoulderLeft_X'])
    df['Shoulder_Y'] = df['ShoulderLeft_Y'] - center['ShoulderLeft_Y']
    df['Shoulder_Z'] = df['ShoulderLeft_Z'] - center['ShoulderLeft_Z']
    df['Thumb_X'] = -(df['ThumbLeft_X'] - center['ShoulderLeft_X'])
    df['Thumb_Y'] = df['ThumbLeft_Y'] - center['ShoulderLeft_Y']
    df['Thumb_Z'] = df['ThumbLeft_Z'] - center['ShoulderLeft_Z']
    df['ShoulderRef_X'] = df['Shoulder_X']
    df['ShoulderRef_Y'] = df['Shoulder_Y']
    df['ShoulderRef_Z'] = df['Shoulder_Z']
    df['ShoulderOther_X'] = df['ShoulderRight_X']
    df['ShoulderOther_Y'] = df['ShoulderRight_Y']
    df['ShoulderOther_Z'] = df['ShoulderRight_Z']
    center['ShoulderLeft_X'] -= center['ShoulderLeft_X']
    center['ShoulderLeft_Y'] -= center['ShoulderLeft_Y']
    center['ShoulderLeft_Z'] -= center['ShoulderLeft_Z']

vectorX = df['Wrist_X'] 
vectorY = df['Wrist_Y']
vectorZ = df['Wrist_Z'] 
vectors=np.column_stack((vectorX[:-1], vectorY[:-1], vectorZ[:-1]))
#find the radius of the spherical distribution
magnitudes = np.linalg.norm(vectors, axis=1)
largest_magnitude = np.max(magnitudes)
avg_magnitude = np.mean(magnitudes)
med_magnitude = np.median(magnitudes)
radius = largest_magnitude
print(radius)
print(avg_magnitude)
print(med_magnitude)

# golden ratio
phi = (1 + np.sqrt(5)) / 2

# vertices of a unit icosahedron
icosahedron_vertices = np.array([
    [0, 1, phi],
    [0, -1, phi],
    [0, 1, -phi],
    [0, -1, -phi],
    [1, phi, 0],
    [-1, phi, 0],
    [1, -phi, 0],
    [-1, -phi, 0],
    [phi, 0, 1],
    [-phi, 0, 1],
    [phi, 0, -1],
    [-phi, 0, -1]
])

# scaling by the largest magnitude of vector_normalized (from prev calc)
factorNorm = radius / phi
icosahedron_vertices *= factorNorm

    # Assuming icosahedron_vertices and center are defined
center = np.array([0, 0, 0])
distances = cdist(icosahedron_vertices, icosahedron_vertices)

# Find the 5 nearest neighbors for each vertex (excluding itself)
# argsort sorts distances, so [:, 1:6] skips the first column (distance to itself) and selects the next 5 nearest neighbors.
nearest_neighbors_indices = np.argsort(distances, axis=1)[:, 1:6]
# Sort nearest neighbors indices for consistent processing
nearest_neighbors_indices = np.sort(nearest_neighbors_indices, axis=1)

unique_pyramids_set = set()
for vertex_idx, neighbors_idx in enumerate(nearest_neighbors_indices):
    vertex = icosahedron_vertices[vertex_idx]
    neighbors = icosahedron_vertices[neighbors_idx]

    # Generate combinations of neighbors
    neighbor_combinations = itertools.combinations(neighbors_idx, 2)

    for combination_idx in neighbor_combinations:
        combination = icosahedron_vertices[np.array([vertex_idx, *combination_idx])]
        combination = combination[np.lexsort(combination.T)]  # Sort vertices of the pyramid

        # Calculate distances
        dist_01 = np.linalg.norm(combination[0] - combination[1])
        dist_12 = np.linalg.norm(combination[1] - combination[2])
        dist_20 = np.linalg.norm(combination[2] - combination[0])
        
        # Check for equilateral triangle
        mean_dist = np.mean([dist_01, dist_12, dist_20])
        if np.allclose([dist_01, dist_12, dist_20], mean_dist, atol=1e-6):
            # Add sorted combination to the set with the center
            sorted_indices = tuple(sorted([vertex_idx, *combination_idx]))
            unique_pyramids_set.add(sorted_indices)

# Convert back to list of pyramids and prepend the center point
unique_pyramids = []
for indices in unique_pyramids_set:
    pyramid_vertices = icosahedron_vertices[list(indices)]
    pyramid_with_center = np.vstack([center, pyramid_vertices])  # Add center as the apex
    unique_pyramids.append(pyramid_with_center)
        
print(np.shape(unique_pyramids))
array_unique_pyramids = np.array(unique_pyramids)

#Prepare bin centers etc just for example calculations down the road - just for show in summary stats
wristMotionX = df['Wrist_X'] #flip left X to match right for analysis
wristMotionY = df['Wrist_Y']
wristMotionZ = df['Wrist_Z']
    
wristMotionOnly=np.column_stack((wristMotionX, wristMotionY, wristMotionZ))
n_bins = 3
xb, bin_limits, bin_centers = fixing_bin_center(wristMotionOnly, n_bins, center, right)

#outsideCI_perzone = outsideCI_perzone.flatten().astype(int)
title = 'Flagged Participants Probabilities'
colorbar_title = '# participants outside CI'
plot3dIcosahedron(heatDifference_icosahedron[-20:], title, colorbar_title, array_unique_pyramids, nt_ci_data, 0, heatDifference_icosahedron[:20])
plot3dIcosahedron(heatDifference_icosahedron[:20], title, colorbar_title, array_unique_pyramids/2, nt_ci_data, 1, heatDifference_icosahedron[-20:])


plot3dDust(df, 'Sample Neurotypical Wrist Position Data',right)

plot3dRectilinear(stroke_diff_df, Rect_nt_ci_data, 'Rectilinear Model Data')

# Make a grid that shows the average percentage in each zone - organize not by number but by relation to each other- Superior/Poster
#plot3dIcosahedron_towers(outsideCI_perzone, title, colorbar_title, array_unique_pyramids, heatmap_plot)

jointAngles = calc_jointangles(df)
